# THESIS SETUP

## Imports

In [1]:
import pandas as pd
import numpy as np
import os, copy
import pickle
from scipy import stats
from datetime import timedelta, datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D, AveragePooling1D
from keras.layers import LSTM, GRU, RepeatVector, TimeDistributed, Bidirectional, Attention
from tensorflow.keras.layers import BatchNormalization
from keras.initializers import *
import pytz
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.stats.stattools import medcouple
from tcn import TCN, tcn_full_summary

# clear warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

2023-11-19 12:40:31.253153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Utility Functions

In [2]:
 #evaluate prediction and save the accuracy results
def evaluatePrediction(allPred, actData,  dataRange):
    """
    :param allPred:
    :param actData:
    :param dataRange:
    :return:
    """
    modelName=list(allPred.keys())[0]
    accDF=computeAccuracy(actData=actData, predData=allPred[modelName], dataRange=dataRange)
    return accDF

#compute prediction accuracy
def computeAccuracy(actData, predData, dataRange=400):
    """
    :param actData:
    :param predData:
    :param dataRange:
    :return:
    """
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    MAE=[]
    RMSE=[]
    R2=[]
    PC=[]
    MRE=[]

    for i in range(actData.shape[1]):
        curTarget=actData[:,i]
        nonNanInd=~np.isnan(curTarget)
        curTarget=curTarget[nonNanInd]

        curPred=predData[:,i]
        curPred=curPred[nonNanInd]

        nonNanInd = ~np.isnan(curPred)
        curTarget=curTarget[nonNanInd]
        curPred=curPred[nonNanInd]

        if curTarget.shape[0]<=10:
            MAE.append (9999999999)
            MRE.append (9999999999)
            RMSE.append(9999999999)
            R2.append(0)
            PC.append(0)
        else:
            absError = np.abs(curTarget - curPred)
            relabsError = absError / dataRange

            MAE.append(mean_absolute_error(y_true=curTarget, y_pred=curPred))
            MRE.append(np.nanmean(relabsError) * 100)
            RMSE.append(np.sqrt(mean_squared_error(y_true=curTarget, y_pred=curPred)))
            R2.append(r2_score(y_true=curTarget, y_pred=curPred))
            PC.append(stats.pearsonr(curTarget, curPred)[0])

    actFlat=actData.flatten()
    nonNanInd = ~np.isnan(actFlat)
    actFlat = actFlat[nonNanInd]
    predFlat=predData.flatten()
    predFlat = predFlat[nonNanInd]

    nonNanInd=~np.isnan(predFlat)
    actFlat=actFlat[nonNanInd]
    predFlat=predFlat[nonNanInd]

    if curTarget.shape[0] <= 10:
        MAE.append(9999999999)
        MRE.append(9999999999)
        RMSE.append(9999999999)
        R2.append(0)
        PC.append(0)
    else:
        absError = np.abs(actFlat - predFlat)
        relabsError = absError / dataRange

        MAE.append(mean_absolute_error(y_true=actFlat, y_pred=predFlat))
        MRE.append(np.nanmean(relabsError) * 100)
        RMSE.append(np.sqrt(mean_squared_error(y_true=actFlat, y_pred=predFlat)))
        R2.append(r2_score(y_true=actFlat, y_pred=predFlat))
        PC.append(stats.pearsonr(actFlat, predFlat)[0])


    rowLabel=list(range(1, actData.shape[1]+1))
    rowLabel=['Step_'+str(s) for s in rowLabel]
    rowLabel.append('overall')
    accuracy={'Steps': rowLabel, 'MAE': MAE, 'MRE': MRE, 'RMSE':RMSE, 'R2':R2, 'PC':PC}
    accDF=pd.DataFrame(accuracy)

    return accDF

# create new directories
def createDirectory(dirName: str):
    """
    :param dirName: a str inidicating the entire path for the directory to be created
    :return:
    """
    if not os.path.exists(dirName):
        try:
            os.makedirs(dirName)
        except:
            print(' directory can not be created: ' + dirName)
            exit(-1)
    return

# read data from csv file
def readCSVdata(filePath: str, reqCol=None, dateCol=None, rowsToSkip=None):
    """
    read data from raw csv file and do initial processing if necessary
    :param filePath: a string for complete path of the file
    :param reqCol: list containing name of the columns to read. read all columns if None
    :param dateCol: a string indicating the name of Time col, format the the datetime if not None
    :param rowsToSkip: how many rows to skip
    :return dataDF: data frame
    """

    dataDF = pd.read_csv(filepath_or_buffer=filePath, usecols=reqCol, skiprows=rowsToSkip, na_values = "#DIV/0!")
    if dateCol:
        dataDF[dateCol] = pd.to_datetime(arg=dataDF[dateCol], format="%d/%m/%Y %H:%M")

    return dataDF

# read data from xl file
def readXlsData(filePath, sheetName='Sheet1', reqCol=None, dateCol=None):
    """
    :param filePath:
    :param sheetName:
    :param reqCol:
    :param dateCol:
    :param rowToSkip:
    :return:
    """
    dataDF = pd.read_excel(io=filePath, sheet_name=sheetName, usecols=reqCol)
    if dateCol:
        dataDF[dateCol] = pd.to_datetime(arg=dataDF[dateCol], format="%Y-%m-%d %H:%M:S")

    return dataDF

# save a dataframe to csv format in the disk
def writeDataToCSV(data, outputFile, writeIndex=False, convertToDF=None, indexCol=None):
    """
    :param data: a data frame (or ndarray array) to be saved in the disk
    :param outputFile: full file path
    :param writeIndex: flag indicating whether to write the index col or not
    :param convertToDF: if data is a ndarray, then flag for converting to data frame
    :param indexCol: column that should be set as index col if not None
    :return:
    """

    if convertToDF:
        dataDF = pd.DataFrame(data=data, columns=convertToDF)
        if indexCol:
            dataDF['Time_last_data_sample'] = indexCol
            dataDF.set_index('Time_last_data_sample', inplace=True)

        dataDF.to_csv(path_or_buf=outputFile, index=writeIndex, date_format="%d-%m-%Y %H:%M")
    else:
        data.to_csv(path_or_buf=outputFile, index=writeIndex, date_format="%d-%m-%Y %H:%M")

    return

# save a dataframe to json format in the disk
def writeDataToJSON(data, outputFile, convertToDF=None, indexCol=None, convertDateToString=False):
    """
    :param data:
    :param outputFile:
    :param convertToDF:
    :param indexCol:
    :param convertDateToString:
    :return:
    """
    dataDF=copy.deepcopy(data)
    if convertToDF:
        dataDF = pd.DataFrame(data=data, columns=convertToDF)
        if indexCol:
            dataDF['Pred_start']=indexCol
            dataDF.set_index('Pred_start', inplace=True, drop=True)

    if convertDateToString:
        indexName=dataDF.index.name
        dataDF.reset_index(drop=False, inplace=True)
        dataDF[indexName]=dataDF[indexName].dt.strftime("%d-%m-%Y %H:%M")
        dataDF.set_index(indexName, inplace=True, drop=True)

    dataDF.to_json(path_or_buf=outputFile,  orient='index', indent=4, double_precision=3)

    return

# save the data scaler in the disk
def saveDataScaler(dataScaler, dirName):
    # create the model directory if it does not exist
    createDirectory(dirName=dirName)

    for key in dataScaler:
        scalerFile =os.path.join(dirName, 'scaler_'+str(key)+'.pkl')
        pickle.dump(dataScaler[key], open(scalerFile, 'wb'))

    return

# load the data scaler from the disk
def loadDataScaler(dirName, names):
    # check if model directory exists
    if not os.path.exists(dirName):
        raise ValueError('Model directory does not exists - data scaler can not be loaded')

    dataScaler=dict()

    for key in names:
        scalerFile = os.path.join(dirName, 'scaler_'+str(key)+'.pkl')
        dataScaler[key]=pickle.load(open(scalerFile, 'rb'))

    return dataScaler

# save predicted data both csv and json file
def savePredictedData(allPred, dataSetName, resDir, maxPredStep, histObsTime, is_training):
    stepNames = list(range(1, maxPredStep+1))
    stepNames = ['Step_' + str(s) for s in stepNames]
    if is_training:
        evalType=''
    else:
        evalType=histObsTime[0].strftime('%Y_%b_%d_%H_%M')

    for modelName in allPred.keys():
        if evalType:
            csvFileName = os.path.join(resDir, str(evalType)+'_'+'predicted_data_' + modelName + '_' + dataSetName + '.csv')
            jsonFileName = os.path.join(resDir, str(evalType)+'_'+'predicted_data_' + modelName + '_' + dataSetName + '.json')
        else:
            csvFileName = os.path.join(resDir, 'predicted_data_' + modelName + '_' + dataSetName + '.csv')
            jsonFileName = os.path.join(resDir, 'predicted_data_' + modelName + '_' + dataSetName + '.json')

        writeDataToCSV(data=allPred[modelName], outputFile=csvFileName, writeIndex=True, convertToDF=stepNames, indexCol=histObsTime)
        writeDataToJSON(data=allPred[modelName], outputFile=jsonFileName, convertToDF=stepNames, indexCol=histObsTime, convertDateToString=True)

    return

## Data Processing

In [3]:
# preprocess the data
def datapreparation_chromasun(is_training, expObj, noisyData):
    """
    :param is_training: boolean, True - training
    :param expObj: object of Experimentsetup class
    :return:
    """
    # create results/models directories if they don't exists
    createDirectory(dirName=expObj.resDir)
    createDirectory(dirName=expObj.modelDir)

    # name of the columns expected in the final combined data frame
    reqCol = expObj.getReqColList()  # list of columns required in the data file

    # read site data from csv/xlsx file regardless of training or real time deployment
    fileExtension = os.path.splitext(expObj.site_data_file)[1]
    if fileExtension.lower()=='.csv':
        rawData = readCSVdata(filePath=os.path.join(expObj.dataDir,expObj.site_data_file))
    elif fileExtension.lower()=='.xlsx':
        rawData = readXlsData(filePath=os.path.join(expObj.dataDir,expObj.site_data_file), sheetName='Sheet1', reqCol=None, dateCol=None)
    else:
        raise FileNotFoundError('Unsupported file extension, please use either csv or xlsx file')

    rawData.rename(columns=expObj.colAlias, inplace=True) # rename the col as required
    rawData[expObj.dateCol] = pd.to_datetime(arg=rawData[expObj.dateCol], format=expObj.site_datetime_format) # format the date col

    # if test, remove the data beyond 2 weeks and check if there is any missing data in last nLagDay
    if not is_training:
        lastHistTime= rawData[expObj.dateCol].iloc[-1]
        lastHistHour=lastHistTime.hour
        lastHistMinute=lastHistTime.minute
        if (lastHistMinute>=0) & (lastHistMinute<=29):
            lastHistMinute=0
        else:
            lastHistMinute=30

        lastHistDay=lastHistTime.replace(hour=0, minute=0, second=0)
        histStart=lastHistDay-pd.Timedelta(days=14) # keep only last 2 weeks of data
        histEnd=rawData[expObj.dateCol].iloc[-1]
        reqInd=list(rawData.index[(rawData[expObj.dateCol]>=histStart) & (rawData[expObj.dateCol] <=histEnd)])
        rawData=rawData.loc[reqInd,:]
        rawData.reset_index(inplace=True, drop=True)

        testHistTime=lastHistTime.replace(hour=lastHistHour, minute=lastHistMinute, second=0) # this is required to keep track of time last observed data during deployment
        expObj.timeLastObservation = testHistTime

    else:
        lastHistTime = None  # these variables are set none for training (they only need for testing (when is_training=False))
        lastHistDay = None
        lastHistHour = None
        lastHistMinute = None
        testHistTime=None

    #Compute the flow rate if not available
    if not (expObj.flowrateCol in rawData):
        rawData[expObj.flowrateCol]=rawData['PU_FS']*rawData['HT_FM'] # L/s
        rawData[expObj.flowrateCol]=rawData[expObj.flowrateCol]*0.965 # kg/s

    # remove data for days with number of zero power output/chiller off state during day time
    # is higher than specified threshold(which means outlier day)
    rawData[expObj.deltaTempCol] = rawData[expObj.outletTempCol] - rawData[expObj.inletTempCol]
    rawData[expObj.targetCol]=rawData[expObj.deltaTempCol] * rawData[expObj.flowrateCol] * expObj.waterSpecHeat
    nonZeroDays = removeZeroDays(rawData=rawData, expObj=expObj)
    rawData = getDataForGoodDays(rawDF=rawData, qcPassedDays=nonZeroDays, expObj=expObj)
    chillerOnDays = removeChillerOffDays(rawData=rawData, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay)
    rawData = getDataForGoodDays(rawDF=rawData, qcPassedDays=chillerOnDays, expObj=expObj)
    rawData.drop(columns=[expObj.deltaTempCol, expObj.targetCol], inplace=True) # remove previously added column

    # set flow rate beyond start/end window as zero
    if expObj.windowStart:
        rawData['hour']=rawData[expObj.dateCol].dt.hour
        startHour=int(expObj.windowStart[0:2])
        endHour=int(expObj.windowEnd[0:2])
        reqInd=list(rawData.index[(rawData['hour'] <startHour) | (rawData['hour'] >endHour)])
        rawData.loc[reqInd, [expObj.flowrateCol]]=0
        rawData.drop(columns=['hour'], inplace=True)

    # For Chromasun data set, remove the data after 2019 since they look like unusual
    if (len(expObj.dataExclusion)>0) & is_training:
        exclusionStart=pd.to_datetime(arg=expObj.dataExclusion[0], format="%d-%m-%Y")
        if len(expObj.dataExclusion)==1:
            reqInd = list(rawData.index[(rawData[expObj.dateCol] <exclusionStart)])
        else:
            exclusionEnd=pd.to_datetime(arg=expObj.dataExclusion[1], format="%d-%m-%Y")
            exclusionEnd.replace(hour=23, minute=59, second=59)
            reqInd = list(rawData.index[(rawData[expObj.dateCol] <exclusionStart) | (rawData[expObj.dateCol] > exclusionEnd)])

        rawData=rawData.loc[reqInd,:]
        rawData.reset_index(inplace=True, drop=True)

    # find info on missing data on daily basis and then keep the data only for days that passed quality check
    qcPassedDays=dailyMissingInfo(rawData=rawData, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay)
    goodDF=getDataForGoodDays(rawDF=rawData, qcPassedDays=qcPassedDays, expObj=expObj)

    # replace missing values
    goodDF=replaceMissingData(rawData=goodDF, qcPassedDays=qcPassedDays, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay, doFill=True)
    goodDF[expObj.deltaTempCol] = goodDF[expObj.outletTempCol] - goodDF[expObj.inletTempCol]

    # replace bad flow rate and temp
    goodDF =replaceBadTempFL(dataDF=goodDF, expObj=expObj) # process neg temp and low flow rate before computation of power output

    # again check flow rate and set flow rate beyond start/end window as zero
    if expObj.windowStart:
        goodDF['hour'] = goodDF[expObj.dateCol].dt.hour
        startHour = int(expObj.windowStart[0:2])
        endHour = int(expObj.windowEnd[0:2])
        reqInd = list(goodDF.index[(goodDF['hour'] < startHour) | (goodDF['hour'] > endHour)])
        goodDF.loc[reqInd, [expObj.flowrateCol, expObj.deltaTempCol]] = 0
        goodDF.drop(columns=['hour'], inplace=True)

    # compute the power output data
    goodDF[expObj.targetCol]=goodDF[expObj.deltaTempCol]*goodDF[expObj.flowrateCol]*expObj.waterSpecHeat
    goodDF[expObj.targetCol]=goodDF[expObj.targetCol].abs() # to set -0 as 0

    # Apply outlier detection method and replace the outliers data
    cleanDF = processOutliers(dataDF=goodDF, expObj=expObj, checkPowerThreshold = True)

    odm=list(expObj.outliersDetMethods.keys())[0]
    outliersInd = list(cleanDF.index[(cleanDF[odm] == True)])
    cleanDF.loc[outliersInd, expObj.targetCol] = np.nan
    cleanDF.fillna(method='ffill', inplace=True)
    cleanDF.fillna(method='bfill', inplace=True)

    # Again check the flow rate and set flow rate beyond start/end window as zero
    cleanDF['hour'] = cleanDF[expObj.dateCol].dt.hour
    if expObj.windowStart:
        startHour = int(expObj.windowStart[0:2])
        endHour = int(expObj.windowEnd[0:2])
        reqInd = list(cleanDF.index[(cleanDF['hour'] < startHour) | (cleanDF['hour'] > endHour)])
        cleanDF.loc[reqInd, [expObj.flowrateCol, expObj.deltaTempCol, expObj.targetCol]] = 0
        cleanDF.drop(columns=['hour'], inplace=True)

    # check the quality again and remove data for bad days
    qcPassedDays = removeZeroDays(rawData=cleanDF, expObj=expObj)
    cleanDF = getDataForGoodDays(rawDF=cleanDF, qcPassedDays=qcPassedDays, expObj=expObj)

    # resample data as required
    if expObj.chillerStateCol in expObj.histVar:
        cleanDF = cleanDF[[expObj.dateCol, expObj.chillerStateCol, expObj.flowrateCol, expObj.inletTempCol,
                           expObj.outletTempCol, expObj.deltaTempCol, expObj.ambTempCol, expObj.targetCol]]
    else:
        cleanDF = cleanDF[[expObj.dateCol, expObj.flowrateCol, expObj.inletTempCol,
                           expObj.outletTempCol, expObj.deltaTempCol, expObj.ambTempCol, expObj.targetCol]]

    resampledData = getResampledData(rawData=cleanDF, qcPassedDays=qcPassedDays, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay)

    # if test, check if the number of samples is equal to nLags
    if not is_training:
        if resampledData.shape[0]<expObj.nLags:
            raise ValueError ('Sufficient historical (with good quality) is not available for input to the model')
    
    # combine historical GHI, DNI data
    if ('GHI' in expObj.histVar) | ('DNI' in expObj.histVar):
        combData = appendSolarRadiationData(processedDF=resampledData, expObj=expObj, is_training=is_training,
                                            lastHistDay=lastHistDay, lastHistHour=lastHistHour, lastHistMinute=lastHistMinute)
    else:
        combData=resampledData

    if is_training:
        expObj.checkColumns(data=combData, replacePredVar=True)
        if expObj.doShuffle:
            combData = dataShuffle(orgData=combData, dateCol=expObj.dateCol, randSeed=expObj.randSeed) # shuffle the days randomly if requested only for training

        # split the data into train-validation-test sets
        expObj.getSplitIndex(dataDF=combData, discardLastTestDay=True)  # set the integer index (loc) of the data samples for training, validation, and testing
        # compute data range data range
        expObj.dataRange = combData[expObj.targetCol].max() - combData[expObj.targetCol].min()
        # train and save data normalizer
        expObj.dataScaler = expObj.getDataScaler(orgData=combData, dataLoc=expObj.trVdLoc)  # configure the data scaler
        saveDataScaler(dataScaler=expObj.dataScaler, dirName=expObj.modelDir)

    else:
        lastCombTime=combData[expObj.dateCol].iloc[-1]
        lastCombHour=lastCombTime.hour
        lastCombMinute=lastCombTime.minute
        # find same hour and mintue from history if we don't have quality data at last hour/minute current day
        if (lastCombHour!=lastHistHour) | (lastCombMinute !=lastHistMinute):
            combData['Hour']=combData[expObj.dateCol].dt.hour
            combData['Minute'] = combData[expObj.dateCol].dt.minute
            tempInd=list(combData.index[(combData['Hour'] == lastHistHour) & (combData['Minute'] == lastHistMinute)])
            tempInd=tempInd[-1]
            validInd=list(range(0, tempInd+1))
            combData=combData.loc[validInd,:]
            combData.drop(columns=['Hour', 'Minute'], inplace=True)
        # index for test data
        expObj.testInd = [combData.shape[0], combData.shape[0]]  # starting index of the first step out of maxPredStep
        expObj.testLoc = list(range(expObj.testInd[0], expObj.testInd[1] + 1))

        # check if missing data in nLags
        inputVarNames=[expObj.dateCol, expObj.targetCol]
        inputVarNames.extend(expObj.histVar)
        combData=combData.loc[:,inputVarNames]
        histDF=combData.tail(expObj.nLags)
        checkNaNValue(df=histDF, expObj=expObj, isHistory=True)

        # combine predicted data for GHI, DNI and Amb temp from different file
        if (expObj.predVar):
            futureData = getPredictedData(expObj=expObj)  # read the predicted future data as required
            # check if future data has enough rows and whether is there any missing values
            checkFutureData(df=futureData, expObj=expObj, lastHistDay=lastHistDay, lastHistHour=lastHistHour, lastHistMinute=lastHistMinute)
            # merge the future data with historical data
            combData = pd.merge(combData, futureData, how='outer', left_on=expObj.dateCol, right_on=expObj.dateCol)

        # check if all the necessary columns exists, if not raise exception during testing/deployment
        expObj.checkColumns(data=combData, replacePredVar=False)

        # load the data scaler
        inputVarNames.extend(expObj.predVar)
        inputVarNames.extend(expObj.calendarVar)
        inputVarNames[inputVarNames.index(expObj.targetCol)]='target'
        inputVarNames.remove(expObj.dateCol)
        expObj.dataScaler = loadDataScaler(dirName=expObj.modelDir, names=inputVarNames)

    #keep only the requied columns
    combData=combData.loc[:, reqCol]
    
    if noisyData:
        #adding gaussian noise
        mu = 0

        std_amb_temp = np.std(combData["pred_Amb_temp"]) * 0.2
        amb_temp_noise = np.random.normal(mu, std_amb_temp, size = len(combData["pred_Amb_temp"]))
        new_Amb_temp = combData["pred_Amb_temp"] + amb_temp_noise
        combData["pred_Amb_temp"] = new_Amb_temp

        std_dni = np.std(combData["pred_DNI"]) * 0.2
        std_ghi = np.std(combData["pred_GHI"]) * 0.2

        ghi_noise = np.random.normal(mu, std_ghi, size = len(combData["pred_GHI"]))
        dni_noise = np.random.normal(mu, std_dni, size = len(combData["pred_DNI"]))

        new_ghi = combData["pred_GHI"] + ghi_noise
        new_dni = combData["pred_DNI"] + dni_noise

        for i in combData.index:
            if combData["pred_GHI"][i] == 0 or new_ghi[i] < 0:
                new_ghi[i] = 0
            if combData["pred_DNI"][i] == 0 or new_dni[i] < 0:
                new_dni[i] = 0

        combData["pred_GHI"] = new_ghi
        combData["pred_DNI"] = new_dni
        
    return combData

# preprocess the data
def datapreparation_greenland(is_training, expObj, noisyData):
    """
    :param is_training: boolean, True - training
    :param expObj: object of Experimentsetup class
    :return:
    """
    # create results/models directories if they don't exists
    createDirectory(dirName=expObj.resDir)
    createDirectory(dirName=expObj.modelDir)

    # name of the columns expected in the final combined data frame
    reqCol = expObj.getReqColList()  # list of columns required in the data file

    # read site data from csv/xlsx file regardless of training or real time deployment
    fileExtension = os.path.splitext(expObj.site_data_file)[1]

    rawData = readCSVdata(filePath=os.path.join(expObj.dataDir,expObj.site_data_file))
    convert_dict = {
        'T2 SF - Collector In' : float,
        'T1 SF - Collector Out' : float, 
        'Amb Temp' : float,
        'FM1 9 l/s span - Collector Flow' : float
    }
    rawData = rawData.astype(convert_dict)

    rawData.rename(columns=expObj.colAlias, inplace=True) # rename the col as required
    rawData[expObj.dateCol] = pd.to_datetime(arg=rawData[expObj.dateCol], format=expObj.site_datetime_format) # format the date col

    # if test, remove the data beyond 2 weeks and check if there is any missing data in last nLagDay
    if not is_training:
        lastHistTime= rawData[expObj.dateCol].iloc[-1]
        lastHistHour=lastHistTime.hour
        lastHistMinute=lastHistTime.minute
        if (lastHistMinute>=0) & (lastHistMinute<=29):
            lastHistMinute=0
        else:
            lastHistMinute=30

        lastHistDay=lastHistTime.replace(hour=0, minute=0, second=0)
        histStart=lastHistDay-pd.Timedelta(days=14) # keep only last 2 weeks of data
        histEnd=rawData[expObj.dateCol].iloc[-1]
        reqInd=list(rawData.index[(rawData[expObj.dateCol]>=histStart) & (rawData[expObj.dateCol] <=histEnd)])
        rawData=rawData.loc[reqInd,:]
        rawData.reset_index(inplace=True, drop=True)

        testHistTime=lastHistTime.replace(hour=lastHistHour, minute=lastHistMinute, second=0) # this is required to keep track of time last observed data during deployment
        expObj.timeLastObservation = testHistTime

    else:
        lastHistTime = None  # these variables are set none for training (they only need for testing (when is_training=False))
        lastHistDay = None
        lastHistHour = None
        lastHistMinute = None
        testHistTime=None

    # For Chromasun data set, remove the data after 2019 since they look like unusual
    if (len(expObj.dataExclusion)>0) & is_training:
        exclusionStart=pd.to_datetime(arg=expObj.dataExclusion[0], format="%d-%m-%Y")
        if len(expObj.dataExclusion)==1:
            reqInd = list(rawData.index[(rawData[expObj.dateCol] <exclusionStart)])
        else:
            exclusionEnd=pd.to_datetime(arg=expObj.dataExclusion[1], format="%d-%m-%Y")
            exclusionEnd.replace(hour=23, minute=59, second=59)
            reqInd = list(rawData.index[(rawData[expObj.dateCol] <exclusionStart) | (rawData[expObj.dateCol] > exclusionEnd)])

        rawData=rawData.loc[reqInd,:]
        rawData.reset_index(inplace=True, drop=True)

    # find info on missing data on daily basis and then keep the data only for days that passed quality check
    qcPassedDays=dailyMissingInfo(rawData=rawData, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay)
    goodDF=getDataForGoodDays(rawDF=rawData, qcPassedDays=qcPassedDays, expObj=expObj)

    # replace missing values
    goodDF=replaceMissingData_greenland(rawData=goodDF, qcPassedDays=qcPassedDays, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay, doFill=True)
    goodDF[expObj.deltaTempCol] = goodDF[expObj.outletTempCol] - goodDF[expObj.inletTempCol]

    # replace bad flow rate and temp
    goodDF =replaceBadTempFL_greenland(dataDF=goodDF, expObj=expObj) # process neg temp and low flow rate before computation of power output

    # compute the power output data
    goodDF[expObj.targetCol]=goodDF[expObj.deltaTempCol]*goodDF[expObj.flowrateCol]*expObj.waterSpecHeat
    goodDF[expObj.targetCol]=goodDF[expObj.targetCol].abs() # to set -0 as 0

    # Apply outlier detection method and replace the outliers data
    cleanDF = processOutliers(dataDF=goodDF, expObj=expObj, checkPowerThreshold = True)

    odm=list(expObj.outliersDetMethods.keys())[0]
    outliersInd = list(cleanDF.index[(cleanDF[odm] == True)])
    cleanDF.loc[outliersInd, expObj.targetCol] = np.nan
    cleanDF.fillna(method='ffill', inplace=True)
    cleanDF.fillna(method='bfill', inplace=True)

    # check the quality again and remove data for bad days
    qcPassedDays = removeZeroDays(rawData=cleanDF, expObj=expObj)
    cleanDF = getDataForGoodDays(rawDF=cleanDF, qcPassedDays=qcPassedDays, expObj=expObj)

    # resample data as required
    if expObj.chillerStateCol in expObj.histVar:
        cleanDF = cleanDF[[expObj.dateCol, expObj.chillerStateCol, expObj.flowrateCol, expObj.inletTempCol,
                           expObj.outletTempCol, expObj.deltaTempCol, expObj.ambTempCol, expObj.targetCol]]
    else:
        cleanDF = cleanDF[[expObj.dateCol, expObj.flowrateCol, expObj.inletTempCol,
                           expObj.outletTempCol, expObj.deltaTempCol, expObj.ambTempCol, expObj.targetCol]]

    resampledData = getResampledData(rawData=cleanDF, qcPassedDays=qcPassedDays, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay)

    # if test, check if the number of samples is equal to nLags
    if not is_training:
        if resampledData.shape[0]<expObj.nLags:
            raise ValueError ('Sufficient historical (with good quality) is not available for input to the model')

    # combine historical GHI, DNI data
    if ('GHI' in expObj.histVar) | ('DNI' in expObj.histVar):
        combData = appendSolarRadiationData(processedDF=resampledData, expObj=expObj, is_training=is_training,
                                            lastHistDay=lastHistDay, lastHistHour=lastHistHour, lastHistMinute=lastHistMinute)
    else:
        combData=resampledData
    
    if is_training:
        expObj.checkColumns(data=combData, replacePredVar=True)
        if expObj.doShuffle:
            combData = dataShuffle(orgData=combData, dateCol=expObj.dateCol, randSeed=expObj.randSeed) # shuffle the days randomly if requested only for training

        # split the data into train-validation-test sets
        expObj.getSplitIndex(dataDF=combData, discardLastTestDay=True)  # set the integer index (loc) of the data samples for training, validation, and testing
        # compute data range data range
        expObj.dataRange = combData[expObj.targetCol].max() - combData[expObj.targetCol].min()
        # train and save data normalizer
        expObj.dataScaler = expObj.getDataScaler(orgData=combData, dataLoc=expObj.trVdLoc)  # configure the data scaler
        saveDataScaler(dataScaler=expObj.dataScaler, dirName=expObj.modelDir)

    else:
        lastCombTime=combData[expObj.dateCol].iloc[-1]
        lastCombHour=lastCombTime.hour
        lastCombMinute=lastCombTime.minute
        # find same hour and mintue from history if we don't have quality data at last hour/minute current day
        if (lastCombHour!=lastHistHour) | (lastCombMinute !=lastHistMinute):
            combData['Hour']=combData[expObj.dateCol].dt.hour
            combData['Minute'] = combData[expObj.dateCol].dt.minute
            tempInd=list(combData.index[(combData['Hour'] == lastHistHour) & (combData['Minute'] == lastHistMinute)])
            tempInd=tempInd[-1]
            validInd=list(range(0, tempInd+1))
            combData=combData.loc[validInd,:]
            combData.drop(columns=['Hour', 'Minute'], inplace=True)
        # index for test data
        expObj.testInd = [combData.shape[0], combData.shape[0]]  # starting index of the first step out of maxPredStep
        expObj.testLoc = list(range(expObj.testInd[0], expObj.testInd[1] + 1))

        # check if missing data in nLags
        inputVarNames=[expObj.dateCol, expObj.targetCol]
        inputVarNames.extend(expObj.histVar)
        combData=combData.loc[:,inputVarNames]
        histDF=combData.tail(expObj.nLags)
        checkNaNValue(df=histDF, expObj=expObj, isHistory=True)

        # combine predicted data for GHI, DNI and Amb temp from different file
        if (expObj.predVar):
            futureData = getPredictedData(expObj=expObj)  # read the predicted future data as required
            # check if future data has enough rows and whether is there any missing values
            checkFutureData(df=futureData, expObj=expObj, lastHistDay=lastHistDay, lastHistHour=lastHistHour, lastHistMinute=lastHistMinute)
            # merge the future data with historical data
            combData = pd.merge(combData, futureData, how='outer', left_on=expObj.dateCol, right_on=expObj.dateCol)

        # check if all the necessary columns exists, if not raise exception during testing/deployment
        expObj.checkColumns(data=combData, replacePredVar=False)

        # load the data scaler
        inputVarNames.extend(expObj.predVar)
        inputVarNames.extend(expObj.calendarVar)
        inputVarNames[inputVarNames.index(expObj.targetCol)]='target'
        inputVarNames.remove(expObj.dateCol)
        expObj.dataScaler = loadDataScaler(dirName=expObj.modelDir, names=inputVarNames)

    #keep only the requied columns
    combData=combData.loc[:, reqCol]
    
    if noisyData:
        #adding gaussian noise
        mu = 0

        std_amb_temp = np.std(combData["pred_Amb_temp"]) * 0.2
        amb_temp_noise = np.random.normal(mu, std_amb_temp, size = len(combData["pred_Amb_temp"]))
        new_Amb_temp = combData["pred_Amb_temp"] + amb_temp_noise
        combData["pred_Amb_temp"] = new_Amb_temp

        std_dni = np.std(combData["pred_DNI"]) * 0.2
        std_ghi = np.std(combData["pred_GHI"]) * 0.2

        ghi_noise = np.random.normal(mu, std_ghi, size = len(combData["pred_GHI"]))
        dni_noise = np.random.normal(mu, std_dni, size = len(combData["pred_DNI"]))

        new_ghi = combData["pred_GHI"] + ghi_noise
        new_dni = combData["pred_DNI"] + dni_noise

        for i in combData.index:
            if combData["pred_GHI"][i] == 0 or new_ghi[i] < 0:
                new_ghi[i] = 0
            if combData["pred_DNI"][i] == 0 or new_dni[i] < 0:
                new_dni[i] = 0

        combData["pred_GHI"] = new_ghi
        combData["pred_DNI"] = new_dni

    return combData

# shuffle the data if required
def dataShuffle(orgData, dateCol, randSeed):
    """
    :param orgData:
    :param dateCol:
    :param randSeed:
    :return:
    """
    actTimestamp=orgData[dateCol].values

    dateVec=copy.deepcopy(orgData[dateCol])
    dateVec=dateVec.apply(lambda x: x.replace(hour=0, minute=0))
    unqDays=dateVec.unique()
    np.random.seed(randSeed)
    unqDays=list(np.random.permutation(unqDays))

    newIndex=[]

    for day in unqDays:
        dayStart=pd.to_datetime(day)
        dayEnd=dayStart.replace(hour=23, minute=59)
        reqInd = list(orgData.index[(orgData[dateCol] >= dayStart) & (orgData[dateCol] <= dayEnd)])
        reqInd.sort()

        newIndex.extend(reqInd)

    orgData=orgData.reindex(newIndex)
    orgData.reset_index(inplace=True, drop=True)

    orgData.loc[:, dateCol]=actTimestamp # shuffle the data but keep the timestamp same as in the org data

    return orgData

# remove days if number of zeros exceeds the threshold percentage
def removeZeroDays(rawData, expObj):
    """
    :param rawData:
    :param expObj:
    :param is_training:
    :param lastHistDay:
    :return:
    """
    dfCopy = copy.deepcopy(rawData)

    startDate = dfCopy[expObj.dateCol].iloc[0]
    endDate = dfCopy[expObj.dateCol].iloc[-1]

    # number of sampels expected
    timeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"),
                              end=pd.to_datetime(arg=expObj.windowEnd, format="%H:%M"), freq=str(expObj.baseResolution) + 'min')
    sampWithinWindow = timeRange.shape[0]

    # dict to store info on missing data
    missInfo = {'Day': [], 'Expected daytime samples': [], 'Available daytime samples': [], 'Zero daytime samples': [],
                'Zero daytime percentage': [], 'Data quality': []}

    # get  zero valued data information
    startDate=startDate.replace(hour=0, minute=0, second=0)
    endDate=endDate.replace(hour=0, minute=0, second=0)
    curDay = startDate
    while (curDay <= endDate):
        quality = 'bad'
        dayFinished = curDay.replace(hour=23, minute=59, second=59)
        dailyDF = dfCopy.loc[(dfCopy[expObj.dateCol] >= curDay) & (dfCopy[expObj.dateCol] <= dayFinished)]

        # missing window basis
        startTime = curDay.strftime('%Y-%m-%d') + ' ' + expObj.windowStart
        startTime = pd.to_datetime(arg=startTime, format="%Y-%m-%d %H:%M")

        endTime = curDay.strftime('%Y-%m-%d') + ' ' + expObj.windowEnd
        endTime = pd.to_datetime(arg=endTime, format="%Y-%m-%d %H:%M")

        winDF = dailyDF.loc[(dailyDF[expObj.dateCol] >= startTime) & (dailyDF[expObj.dateCol] <= endTime)]

        zeroInd = list(winDF.index[(winDF[expObj.targetCol] <= 0)])
        zeroPercent = (len(zeroInd) * 100) / sampWithinWindow

        if (winDF.shape[0]>0) & (zeroPercent <= expObj.percentZeroThreshold):
            quality = 'ok'

        missInfo['Day'].append(curDay)

        missInfo['Expected daytime samples'].append(sampWithinWindow)
        missInfo['Available daytime samples'].append(winDF.shape[0])
        missInfo['Zero daytime samples'].append(len(zeroInd))
        missInfo['Zero daytime percentage'].append(zeroPercent)

        missInfo['Data quality'].append(quality)

        curDay = curDay + pd.Timedelta(days=1)

    # write info on missing data
    missInfo = pd.DataFrame(missInfo)

    # separate info on days with good data quality
    qcPassed = missInfo.loc[(missInfo['Data quality'] == 'ok')]
    #qcPassed.loc[:,'Dist_successive_days'] = copy.deepcopy(qcPassed['Day'].diff().values)

    return qcPassed

# only keep data for good quality days
def getDataForGoodDays(rawDF, qcPassedDays, expObj, isSiteData=True):
    """
    :param rawDF:
    :param qcPassedDays:
    :param expObj:
    :param isSiteData:
    :return:
    """
    reqDF=pd.DataFrame()
    count=0
    for curDay in qcPassedDays['Day']:
        dayFinished = curDay.replace(second=59, minute=59, hour=23)
        count=count+1
        #print(str(count)+ '    '+str(curDay))
        if isSiteData:
            timeRange = pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.baseResolution) + 'min')
        else:
            timeRange = pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.hist_GHI_DNI_resolution) + 'min')

        tempData=rawDF.loc[rawDF[expObj.dateCol].isin(timeRange)]
        if reqDF.empty:
            reqDF=copy.deepcopy(tempData)
        else:
            reqDF=reqDF.append(tempData, ignore_index=True)

    reqDF.drop_duplicates(subset=expObj.dateCol, keep='last', ignore_index=True, inplace=True)

    return reqDF

# remove days if chiller's off status exceed the threshold percentage
def removeChillerOffDays(rawData, expObj, is_training, lastHistDay=None):
    """
    :param rawData:
    :param expObj:
    :param is_training:
    :param lastHistDay:
    :return:
    """
    dfCopy = copy.deepcopy(rawData)

    startDate = dfCopy[expObj.dateCol].iloc[0]
    endDate = dfCopy[expObj.dateCol].iloc[-1]

    # number of sampels expected
    timeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"),
                              end=pd.to_datetime(arg=expObj.windowEnd, format="%H:%M"), freq=str(expObj.baseResolution) + 'min')
    sampWithinWindow = timeRange.shape[0]

    # dict to store info on missing data
    missInfo = {'Day': [], 'Expected daytime samples': [], 'Available daytime samples': [], 'Zero daytime samples': [],
                'Zero daytime percentage': [], 'Data quality': []}

    # get  zero valued data information
    startDate=startDate.replace(hour=0, minute=0, second=0)
    endDate=endDate.replace(hour=0, minute=0, second=0)
    curDay = startDate
    while (curDay <= endDate):
        quality = 'bad'
        dayFinished = curDay.replace(second=59, minute=59, hour=23)
        dailyDF = dfCopy.loc[(dfCopy[expObj.dateCol] >= curDay) & (dfCopy[expObj.dateCol] <= dayFinished)]

        # missing window basis
        startTime = curDay.strftime('%Y-%m-%d') + ' ' + expObj.windowStart
        startTime = pd.to_datetime(arg=startTime, format="%Y-%m-%d %H:%M")

        endTime = curDay.strftime('%Y-%m-%d') + ' ' + expObj.windowEnd
        endTime = pd.to_datetime(arg=endTime, format="%Y-%m-%d %H:%M")

        winDF = dailyDF.loc[(dailyDF[expObj.dateCol] >= startTime) & (dailyDF[expObj.dateCol] <= endTime)]

        zeroInd = list(winDF.index[(winDF[expObj.chillerStateCol] <= 0)])
        zeroPercent = (len(zeroInd) * 100) / sampWithinWindow

        if (winDF.shape[0]>0) & (zeroPercent <= expObj.chillerOffThreshold) & (winDF[expObj.targetCol].max()>=expObj.minPowerThreshold):
            quality = 'ok'

        if (not is_training) & (quality=='bad') & (curDay==lastHistDay): # condition added by considering the situation for practical deployment
            finalTime=dfCopy[expObj.dateCol].iloc[-1]
            if finalTime<endTime:
                quality='ok' # for final day: if we don't have data until the end boundary of the window, we skip quality check for that day

        missInfo['Day'].append(curDay)

        missInfo['Expected daytime samples'].append(sampWithinWindow)
        missInfo['Available daytime samples'].append(winDF.shape[0])
        missInfo['Zero daytime samples'].append(len(zeroInd))
        missInfo['Zero daytime percentage'].append(zeroPercent)

        missInfo['Data quality'].append(quality)

        curDay = curDay + pd.Timedelta(days=1)

    # write info on missing data
    missInfo = pd.DataFrame(missInfo)

    # separate info on days with good data quality
    qcPassed = missInfo.loc[(missInfo['Data quality'] == 'ok')]
    #qcPassed.loc[:,'Dist_successive_days'] = copy.deepcopy(qcPassed['Day'].diff().values)

    return qcPassed

# find info on missing data for each day
def dailyMissingInfo(rawData, expObj, is_training, lastHistDay=None):
    """
    :param rawData:
    :param expObj:
    :param is_training:
    :param lastHistDay:
    :return:
    """
    dfCopy = copy.deepcopy(rawData)

    startDate = dfCopy[expObj.dateCol].iloc[0]
    endDate = dfCopy[expObj.dateCol].iloc[-1]
    finalTime = dfCopy[expObj.dateCol].iloc[-1]

    # drop the duplicate rows
    dfCopy.drop_duplicates(subset=expObj.dateCol, keep='last', ignore_index=True, inplace=True)

    # set the index as the dateCol
    dfCopy.set_index(expObj.dateCol, inplace=True)

    # drop the row if nan in all column except date col
    dfCopy.dropna(how='all', inplace=True)

    # drop nan in both inlet and outlet temp col
    dfCopy.dropna(how='all', subset=[expObj.inletTempCol, expObj.outletTempCol], inplace=True)

    # compute samples on daily and window basis
    dfCopy.reset_index(drop=False,inplace=True)

    # delete data with undesired timestamp
    timeDF = pd.date_range(start=startDate, end=endDate, freq=str(expObj.baseResolution) + 'min')
    timeDF = pd.DataFrame(timeDF, columns=[expObj.dateCol])
    dfCopy = pd.merge(timeDF, dfCopy, how='inner', left_on=expObj.dateCol, right_on=expObj.dateCol)
    #dfCopy.drop(columns='Time', inplace=True)

    # number of sampels expected
    sampPerDay=int((24*60)/expObj.baseResolution)
    timeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"), end=pd.to_datetime(arg=expObj.windowEnd, format="%H:%M"), freq=str(expObj.baseResolution) + 'min')
    sampWithinWindow=timeRange.shape[0]

    # dict to store info on missing data
    missInfo = {'Day':[], 'Expected samples':[], 'Available samples':[], 'Missing samples': [], 'Missing overall percentage':[],
                'Expected daytime samples':[], 'Available daytime samples':[], 'Missing daytime samples':[], 'Missing daytime percentage':[], 'Data quality':[]}

    # get  missing data information
    startDate=startDate.replace(hour=0, minute=0, second=0)
    endDate=endDate.replace(hour=0, minute=0, second=0)
    curDay=startDate
    while(curDay<=endDate):
        quality='ok'
        dayFinished=curDay.replace(second=59, minute=59, hour=23)
        dailyDF=dfCopy.loc[(dfCopy[expObj.dateCol]>=curDay ) & (dfCopy[expObj.dateCol] <= dayFinished)]

        # missing daily basis
        dailySamples=dailyDF.shape[0]
        # if dailySamples<sampPerDay:
        #     print('Found missing data')

        missingSamples=sampPerDay-dailySamples
        overallMissPercent=(missingSamples*100)/sampPerDay


        # missing window basis
        startTime = curDay.strftime('%Y-%m-%d')+' '+expObj.windowStart
        startTime=pd.to_datetime(arg=startTime, format="%Y-%m-%d %H:%M")

        endTime=curDay.strftime('%Y-%m-%d')+' '+expObj.windowEnd
        endTime=pd.to_datetime(arg=endTime, format="%Y-%m-%d %H:%M")

        winDF=dailyDF.loc[(dailyDF[expObj.dateCol]>=startTime ) & (dailyDF[expObj.dateCol] <= endTime)]

        windowSamples=winDF.shape[0]
        windowMissing=sampWithinWindow-windowSamples
        windowMissPercent=(windowMissing*100)/sampWithinWindow

        if (windowMissPercent>expObj.percentMissingThreshold):
            quality='bad'
        if (not is_training) & (quality=='bad') & (curDay==lastHistDay): # condition added by considering the situation for practical deployment
            if finalTime<startTime:
                quality='ok'
            elif (finalTime>=startTime) & (finalTime<=endTime): # compute revised missing percentage
                newTimeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"),
                                                 end=pd.to_datetime(arg=finalTime.strftime('%H:%M'), format="%H:%M"), freq=str(expObj.baseResolution) + 'min')
                newSampWithinWindow = newTimeRange.shape[0]
                newEndTime = finalTime

                winDF = dailyDF.loc[(dailyDF[expObj.dateCol] >= startTime) & (dailyDF[expObj.dateCol] <= newEndTime)]
                windowSamples = winDF.shape[0]
                windowMissing = newSampWithinWindow - windowSamples
                windowMissPercent = (windowMissing * 100) / newSampWithinWindow
                if (windowMissPercent > expObj.percentMissingThreshold):
                    quality = 'bad'
                else:
                    quality = 'ok'

        missInfo['Day'].append(curDay)
        missInfo['Expected samples'].append(sampPerDay)
        missInfo['Available samples'].append(dailySamples)
        missInfo['Missing samples'].append(missingSamples)
        missInfo['Missing overall percentage'].append(overallMissPercent)

        missInfo['Expected daytime samples'].append(sampWithinWindow)
        missInfo['Available daytime samples'].append(windowSamples)
        missInfo['Missing daytime samples'].append(windowMissing)
        missInfo['Missing daytime percentage'].append(windowMissPercent)

        missInfo['Data quality'].append(quality)

        curDay=curDay + pd.Timedelta(days=1)

    # write info on missing data
    missInfo=pd.DataFrame(missInfo)

    # separate info on days with good data quality
    qcPassed = missInfo.loc[(missInfo['Data quality']=='ok')]

    return qcPassed

# replace missing data for good quality days
def replaceMissingData(rawData, qcPassedDays, expObj, is_training, lastHistDay=None, doFill=True):
    dayCount=0
    for curDay in qcPassedDays['Day']:
        if (not is_training) & (curDay==lastHistDay):
            dayFinished=rawData[expObj.dateCol].iloc[-1]
        else:
            dayFinished = curDay.replace(hour=23, minute=59, second=59)

        if dayCount>0:
            timeRange=timeRange.append(pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.baseResolution) + 'min'))
        else:
            timeRange = pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.baseResolution) + 'min')

        dayCount=dayCount+1

    # insert empty row for each missing sample
    x = set(timeRange.values)
    y = set(rawData[expObj.dateCol].values)
    missingTime = list(x.difference(y))
    for z in missingTime:
        loc = timeRange.get_loc(z)
        upperDF = rawData[0:loc]  # slice the upper half of the dataframe
        lowerDF = rawData[loc:]  # slice the lower half of the dataframe
        upperDF = upperDF.append(pd.Series(), ignore_index=True)
        upperDF.loc[loc, expObj.dateCol] = timeRange[loc]
        # Concat the two dataframes
        rawData = pd.concat([upperDF, lowerDF], ignore_index=True)

    if doFill:
        for col in rawData:
            if col==expObj.dateCol:
                continue

            if col==expObj.ambTempCol:
                rawData[col] = rawData[col].fillna(method='bfill', inplace=False)
                rawData[col] = rawData[col].fillna(method='ffill', inplace=False)
            else:
                rawData[col] = rawData[col].fillna(method='ffill', inplace=False)
                rawData[col] = rawData[col].fillna(method='bfill', inplace=False)

    return rawData

# replace bad delta temp and flow rate data
def replaceBadTempFL(dataDF, expObj):
    """
    :param dataDF:
    :param expObj:
    :return:
    """

    # replace all flow rate that are below the threshold with 0
    dataDF.loc[(dataDF[expObj.flowrateCol] <=expObj.minFRthreshold), expObj.flowrateCol] = 0

    # consider the flow rate as outliers if the flow rate is below the threshold for at least certain time (frWindowLen)
    nSamplesWindow=int(expObj.frWindowLen/expObj.baseResolution)
    manualOutlierInd=[]
    for i in dataDF.index:
        #check if flow rate is consistence for the specified time window
        if  dataDF.loc[i, expObj.flowrateCol] > 0:
            lagEnd = i
            lagStart = lagEnd - nSamplesWindow + 1
            prevFR = dataDF.loc[lagStart:lagEnd, expObj.flowrateCol].values
            if ((prevFR <= expObj.minFRthreshold).sum()) >= 1:
                manualOutlierInd.append(i)

    dataDF.loc[manualOutlierInd, [expObj.deltaTempCol, expObj.inletTempCol, expObj.outletTempCol, expObj.flowrateCol]] = 0

    # set inlet, outlet, and delta temp to 0 if flow rate is below threshold
    zeroFlowInd = list(dataDF.index[(dataDF[expObj.flowrateCol]<= expObj.minFRthreshold)])
    dataDF.loc[zeroFlowInd, [expObj.inletTempCol, expObj.outletTempCol, expObj.deltaTempCol]] = 0

    # replace negative temp diff
    reqInd = list(dataDF.index[(dataDF[expObj.deltaTempCol] < 0)])
    dataDF.loc[reqInd, [expObj.deltaTempCol, expObj.inletTempCol, expObj.outletTempCol, expObj.flowrateCol]] = 0

    # check if there still any negative delta temp
    reqInd = list(dataDF.index[(dataDF[expObj.deltaTempCol] < 0)])
    dataDF.loc[reqInd, [expObj.inletTempCol, expObj.outletTempCol, expObj.deltaTempCol]] = np.nan
    dataDF.fillna(method='ffill', inplace=True)

    return dataDF

# replace missing data for good quality days
def replaceMissingData_greenland(rawData, qcPassedDays, expObj, is_training, lastHistDay=None, doFill=True):
    dayCount=0
    for curDay in qcPassedDays['Day']:
        if (not is_training) & (curDay==lastHistDay):
            dayFinished=rawData[expObj.dateCol].iloc[-1]
        else:
            dayFinished = curDay.replace(hour=23, minute=59, second=59)

        if dayCount>0:
            timeRange=timeRange.append(pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.baseResolution) + 'min'))
        else:
            timeRange = pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.baseResolution) + 'min')

        dayCount=dayCount+1

    # insert empty row for each missing sample
    x = set(timeRange.values)
    y = set(rawData[expObj.dateCol].values)
    missingTime = list(x.difference(y))
    for z in missingTime:
        loc = timeRange.get_loc(z)
        upperDF = rawData[0:loc]  # slice the upper half of the dataframe
        lowerDF = rawData[loc:]  # slice the lower half of the dataframe
        upperDF = upperDF.append(pd.Series(), ignore_index=True)
        upperDF.loc[loc, expObj.dateCol] = timeRange[loc]
        # Concat the two dataframes
        rawData = pd.concat([upperDF, lowerDF], ignore_index=True)

    if doFill:
        rawData.fillna(method='ffill', inplace=True)

    return rawData

# replace bad delta temp and flow rate data
def replaceBadTempFL_greenland(dataDF, expObj):
    """
    :param dataDF:
    :param expObj:
    :return:
    """

    # replace all flow rate that are below the threshold with 0
    dataDF.loc[(dataDF[expObj.flowrateCol] <=expObj.minFRthreshold), expObj.flowrateCol] = 0

    # consider the flow rate as outliers if the flow rate is below the threshold for at least certain time (frWindowLen)
    nSamplesWindow=int(expObj.frWindowLen/expObj.baseResolution)
    manualOutlierInd=[]
    for i in dataDF.index:
        #check if flow rate is consistence for the specified time window
        if  dataDF.loc[i, expObj.flowrateCol] > 0:
            lagEnd = i
            lagStart = lagEnd - nSamplesWindow + 1
            prevFR = dataDF.loc[lagStart:lagEnd, expObj.flowrateCol].values
            if ((prevFR <= expObj.minFRthreshold).sum()) >= 1:
                manualOutlierInd.append(i)

    dataDF.loc[manualOutlierInd, [expObj.flowrateCol]] = np.nan
    dataDF.fillna(method='ffill', inplace=True)

    # set inlet, outlet, and delta temp to 0 if flow rate is below threshold
    zeroFlowInd = list(dataDF.index[(dataDF[expObj.flowrateCol]<= expObj.minFRthreshold)])
    dataDF.loc[zeroFlowInd, [expObj.inletTempCol, expObj.outletTempCol, expObj.deltaTempCol]] = 0

    # replace negative temp diff
    reqInd = list(dataDF.index[(dataDF[expObj.deltaTempCol] < 0)])
    dataDF.loc[reqInd, [expObj.inletTempCol, expObj.outletTempCol]] = np.nan
    dataDF.fillna(method='ffill', inplace=True)
    dataDF.loc[reqInd, expObj.deltaTempCol] = dataDF.loc[reqInd, expObj.outletTempCol] - dataDF.loc[reqInd, expObj.inletTempCol]

    # check if there still any negative delta temp
    reqInd = list(dataDF.index[(dataDF[expObj.deltaTempCol] < 0)])
    dataDF.loc[reqInd, [expObj.inletTempCol, expObj.outletTempCol, expObj.deltaTempCol]] = np.nan
    dataDF.fillna(method='ffill', inplace=True)

    return dataDF

# apply automatic outlier detection method
def processOutliers(dataDF, expObj, checkPowerThreshold=True):
    """
    :param dataDF:
    :param expObj:
    :param checkPowerThreshold:
    :return:
    """
    cleanData = copy.deepcopy(dataDF)
    allOutliers={}

    # first detect the outliers manually (i.e consider the power output as outliers if the power output is higher than max threshold)
    nSamplesWindow=int(expObj.frWindowLen/expObj.baseResolution)
    manualOutlierInd=[]
    for i in cleanData.index:
        if checkPowerThreshold:
            if cleanData.loc[i, expObj.targetCol]>expObj.maxPowerThreshold:
                manualOutlierInd.append(i)
                continue

    cleanData.loc[manualOutlierInd, expObj.targetCol]=expObj.maxPowerThreshold

    for odm in expObj.outliersDetMethods:
        #print(str(odm))
        cleanData[odm]=False

        if 'LOF' in odm:
            feat = cleanData[expObj.targetCol].values
            feat = np.expand_dims(feat, axis=1)
            configParam=expObj.outliersDetMethods[odm]
            clf = LocalOutlierFactor(**configParam)
            pred = clf.fit_predict(feat)
            pred = pd.Series(data=pred)
            outlierFlag = pred == -1

        elif odm == 'Robust boxplot':
            q1 = cleanData[expObj.targetCol].quantile(q=0.25)
            q2 = cleanData[expObj.targetCol].quantile(q=0.5)
            q3 = cleanData[expObj.targetCol].quantile(q=0.75)
            Iqd = q3 - q1
            k = 1.5
            a = -4
            b = 3
            dataVec = cleanData[expObj.targetCol].values
            mcValue = medcouple(dataVec)

            if mcValue >= 0:
                lowLimit = q1 - (k * np.exp(a * mcValue) * Iqd)
                upLimit = q3 + (k * np.exp(b * mcValue) * Iqd)
            else:
                lowLimit = q1 - (k * np.exp(-b * mcValue) * Iqd)
                upLimit = q3 + (k * np.exp(-a * mcValue) * Iqd)

            outlierFlag = (cleanData[expObj.targetCol] < lowLimit) | (cleanData[expObj.targetCol] > upLimit)

        else:
            print('invalid outlier detection method')
            exit(-1)

        allOutliers[odm] = copy.deepcopy(outlierFlag.values)

        if manualOutlierInd:
            outlierFlag.loc[manualOutlierInd]=True

        outlierFlag = outlierFlag.values
        cleanData.loc[:, odm] = outlierFlag

    for odm in allOutliers:
        cleanData.loc[:, odm]=allOutliers[odm]

    return cleanData

# resample the data to change the resolution
def getResampledData(rawData, qcPassedDays, expObj, is_training, lastHistDay=None):
    """
    :param rawData:
    :param qcPassedDays:
    :param expObj:
    :param is_training:
    :param lastHistDay:
    :return:
    """
    resampledData = pd.DataFrame(columns=rawData.columns)
    rowCount = -1
    dayCount=0
    for curDay in qcPassedDays['Day']:
        dayCount=dayCount+1
        #print(str(dayCount) + '/' + str(qcPassedDays.shape[0]))
        if (not is_training) & (curDay==lastHistDay):
            dayFinished=rawData[expObj.dateCol].iloc[-1]
        else:
            dayFinished = curDay.replace(second=59, minute=59, hour=23)

        timeRange=pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.resolution) + 'min')

        for curTime in timeRange:

            rowCount = rowCount + 1
            prevTime = curTime - pd.Timedelta(minutes=expObj.resolution)
            reqInd = list(rawData.index[(rawData[expObj.dateCol] > prevTime) & (rawData[expObj.dateCol] <= curTime)])

            chillerStatus=0

            if reqInd:
                curData = rawData.loc[reqInd]
                curData.drop(columns=[expObj.dateCol], inplace=True)
                avgData = curData.mean(axis=0, skipna=True)
                newRow = list(avgData)

                newRow.insert(0, curTime)
                if expObj.chillerStateCol in expObj.histVar:
                    chillerStatus=curData.iloc[-1, curData.columns.get_loc(expObj.chillerStateCol)]

            else:
                newRow = [np.nan] * rawData.shape[1]
                newRow[0] = curTime

            resampledData.loc[rowCount] = newRow
            if expObj.chillerStateCol in expObj.histVar:
                resampledData.loc[rowCount, expObj.chillerStateCol]=chillerStatus


    return resampledData

# append historical GHI/DNI data
def appendSolarRadiationData(processedDF, expObj, is_training, lastHistDay=None, lastHistHour=None, lastHistMinute=None):
    siComponents = ['GHI', 'DNI']
    siData={}
    for comp in siComponents:
        if not (comp in expObj.histVar):
            continue
        if comp=='GHI':
            curFile = os.path.join(expObj.dataDir, expObj.hist_GHI_file)
        else:
            curFile = os.path.join(expObj.dataDir, expObj.hist_DNI_file)

        siData[comp] = pd.read_csv(curFile)
        siData[comp].rename(columns={expObj.hist_GHI_DNI_dateCol: expObj.dateCol, expObj.hist_GHI_DNI_valueCol: comp}, inplace=True)
        siData[comp][expObj.dateCol] = pd.to_datetime(siData[comp][expObj.dateCol], format=expObj.hist_GHI_DNI_datetime_format)

        if expObj.hist_GHI_DNI_timezone.lower()=='utc':
            # convert the timezone from UTC to NSW
            siData[comp][expObj.dateCol] =  siData[comp][expObj.dateCol].apply(lambda x: x.replace(tzinfo=pytz.UTC))  # microsecond=0
            siData[comp][expObj.dateCol] =  siData[comp][expObj.dateCol].apply(lambda x: x.astimezone(pytz.timezone('Australia/NSW')))  # convert utc time to AEST
            siData[comp][expObj.dateCol] =  siData[comp][expObj.dateCol].dt.tz_localize(None)  # unlocalize to avoid complexity aournd DST start/end boundary

        siData[comp].drop_duplicates(subset=expObj.dateCol, keep='last', ignore_index=True, inplace=True)

        # only consider data within the range of site data. this is required especially when is_training=False
        firstDate=processedDF[expObj.dateCol].iloc[0]
        lastDate=processedDF[expObj.dateCol].iloc[-1]
        reqInd=list(siData[comp].index[(siData[comp][expObj.dateCol]>=firstDate) & (siData[comp][expObj.dateCol]<=lastDate)])
        siData[comp]=siData[comp].loc[reqInd,:] # keep only the samples with timestamp within the range of site data

        # get data for good days
        qcPassedDays = getSIqcpassdays(rawData=siData[comp], siComp=comp, expObj=expObj, is_training=is_training, lastHistDay=lastHistDay, lastHistHour=lastHistHour, lastHistMinute=lastHistMinute)

        # separate data for the days that passed QC
        siData[comp] = getDataForGoodDays(rawDF=siData[comp], qcPassedDays=qcPassedDays, expObj=expObj, isSiteData=False)

        # up sample the data if required
        if expObj.hist_GHI_DNI_resolution>=expObj.resolution:
            dayCount = 0
            for curDay in qcPassedDays['Day']:
                dayFinished = curDay.replace(hour=23, minute=59, second=59)
                if (not is_training) & (curDay==lastHistDay):# condition added for last day with incomplete data (for testing case)
                    histEnd=lastHistDay
                    histEnd=histEnd.replace(hour=lastHistHour, minute=lastHistMinute)
                    validInd=list(siData[comp].index[(siData[comp][expObj.dateCol] <= dayFinished)])
                    validInd=validInd[-1]
                    lastValidDate=siData[comp][expObj.dateCol][validInd]

                    dayFinished=min(lastValidDate, histEnd)

                if dayCount > 0:
                    timeRange = timeRange.append(pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.resolution) + 'min'))
                else:
                    timeRange = pd.date_range(start=curDay, end=dayFinished, freq=str(expObj.resolution) + 'min')

                dayCount = dayCount + 1

            timeRange=pd.DataFrame(timeRange, columns=[expObj.dateCol])

            siData[comp] = pd.merge(timeRange, siData[comp], how='left', left_on=expObj.dateCol, right_on=expObj.dateCol, validate='1:1')

    # Filter-out the data outside common time
    combData = copy.deepcopy(processedDF)
    
    for comp in siComponents:        
        # replace the negative value
        negInd = list(siData[comp].index[(siData[comp][comp]<0)])
        siData[comp].loc[negInd, comp] = np.nan
        siData[comp][comp].interpolate(method='linear', inplace=True)
        siData[comp].fillna(method='ffill', inplace=True)
        siData[comp].fillna(method='bfill', inplace=True)

        # join data
        combData = pd.merge(combData, siData[comp], how='inner', left_on=expObj.dateCol, right_on=expObj.dateCol, validate='1:1')

    return combData

# quality check for GHI/DNI data
def getSIqcpassdays(rawData, siComp, expObj, is_training, lastHistDay=None, lastHistHour=None, lastHistMinute=None):
    dfCopy = copy.deepcopy(rawData)

    startDate = dfCopy[expObj.dateCol].iloc[0]
    endDate = dfCopy[expObj.dateCol].iloc[-1]

    # drop the duplicate rows
    dfCopy.drop_duplicates(subset=expObj.dateCol, keep='last', ignore_index=True, inplace=True)

    # set the index as the dateCol
    dfCopy.set_index(expObj.dateCol, inplace=True)

    # drop the row if nan in all column except date col
    dfCopy.dropna(how='all', inplace=True)

    # drop nan in both inlet and outlet temp col
    dfCopy.dropna(how='all', subset=[siComp], inplace=True)

    # compute samples on daily and window basis
    dfCopy.reset_index(drop=False,inplace=True)

    # delete data with undesired timestamp
    timeDF = pd.date_range(start=startDate, end=endDate, freq=str(expObj.hist_GHI_DNI_resolution) + 'min')
    timeDF = pd.DataFrame(timeDF, columns=[expObj.dateCol])
    dfCopy = pd.merge(timeDF, dfCopy, how='inner', left_on=expObj.dateCol, right_on=expObj.dateCol)
    #dfCopy.drop(columns='Time', inplace=True)

    # number of sampels expected
    sampPerDay=int((24*60)/expObj.hist_GHI_DNI_resolution)
    timeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"), end=pd.to_datetime(arg=expObj.windowEnd, format="%H:%M"),
                              freq=str(expObj.hist_GHI_DNI_resolution) + 'min')
    sampWithinWindow=timeRange.shape[0]

    # dict to store info on missing data
    missInfo = {'Day':[], 'Expected samples':[], 'Available samples':[], 'Missing samples': [], 'Missing overall percentage':[],
                'Expected daytime samples':[], 'Available daytime samples':[], 'Missing daytime samples':[], 'Missing daytime percentage':[], 'Data quality':[]}

    # get  missing data information
    startDate=startDate.replace(hour=0, second=0, minute=0)
    endDate=endDate.replace(hour=0, second=0, minute=0)
    curDay=startDate
    while(curDay<=endDate):
        quality='ok'
        dayFinished=curDay.replace(hour=23, minute=59, second=59)
        dailyDF=dfCopy.loc[(dfCopy[expObj.dateCol]>=curDay ) & (dfCopy[expObj.dateCol] <= dayFinished)]


        # missing daily basis
        dailySamples=dailyDF.shape[0]
        # if dailySamples<sampPerDay:
        #     print('Found missing data')

        missingSamples=sampPerDay-dailySamples
        overallMissPercent=(missingSamples*100)/sampPerDay

        # missing window basis
        startTime = curDay.strftime('%Y-%m-%d')+' '+expObj.windowStart
        startTime=pd.to_datetime(arg=startTime, format="%Y-%m-%d %H:%M")

        endTime=curDay.strftime('%Y-%m-%d')+' '+expObj.windowEnd
        endTime=pd.to_datetime(arg=endTime, format="%Y-%m-%d %H:%M")

        winDF=dailyDF.loc[(dailyDF[expObj.dateCol]>=startTime ) & (dailyDF[expObj.dateCol] <= endTime)]

        windowSamples=winDF.shape[0]
        windowMissing=sampWithinWindow-windowSamples
        windowMissPercent=(windowMissing*100)/sampWithinWindow

        if (windowMissPercent>expObj.percentMissingThreshold):
            quality='bad'

        if (not is_training) & (quality=='bad') & (curDay==lastHistDay): # condition added by considering the situation for practical deployment

            finalTime = lastHistDay.replace(hour=lastHistHour, minute=lastHistMinute)
            if finalTime<startTime:
                quality='ok'
            elif (finalTime>=startTime) & (finalTime<=endTime): # compute revised missing percentage
                newTimeRange = pd.date_range(start=pd.to_datetime(arg=expObj.windowStart, format="%H:%M"),
                                                 end=pd.to_datetime(arg=finalTime.strftime('%H:%M'), format="%H:%M"), freq=str(expObj.hist_GHI_DNI_resolution) + 'min')
                newSampWithinWindow = newTimeRange.shape[0]
                newEndTime = finalTime

                winDF = dailyDF.loc[(dailyDF[expObj.dateCol] >= startTime) & (dailyDF[expObj.dateCol] <= newEndTime)]
                windowSamples = winDF.shape[0]
                windowMissing = newSampWithinWindow - windowSamples
                windowMissPercent = (windowMissing * 100) / newSampWithinWindow
                if (windowMissPercent > expObj.percentMissingThreshold):
                    quality = 'bad'
                else:
                    quality = 'ok'

        missInfo['Day'].append(curDay)
        missInfo['Expected samples'].append(sampPerDay)
        missInfo['Available samples'].append(dailySamples)
        missInfo['Missing samples'].append(missingSamples)
        missInfo['Missing overall percentage'].append(overallMissPercent)

        missInfo['Expected daytime samples'].append(sampWithinWindow)
        missInfo['Available daytime samples'].append(windowSamples)
        missInfo['Missing daytime samples'].append(windowMissing)
        missInfo['Missing daytime percentage'].append(windowMissPercent)

        missInfo['Data quality'].append(quality)

        curDay=curDay + pd.Timedelta(days=1)

    # separate info on days with good data quality
    missInfo = pd.DataFrame(missInfo)
    qcPassed = missInfo.loc[(missInfo['Data quality']=='ok')]
    #qcPassed.loc[:,'Dist_successive_days']=copy.deepcopy(qcPassed['Day'].diff().values)

    return qcPassed

# extract the predicted weather data for future
def getPredictedData(expObj):
    """
    :param expObj:
    :return:
    """
    futureData=readCSVdata(filePath=os.path.join(expObj.dataDir, expObj.pred_data_file))
    if not(expObj.dateCol==expObj.pred_data_dateCol):
        futureData.rename(columns={expObj.pred_data_dateCol: expObj.dateCol}, inplace=True)
    futureData[expObj.dateCol] = pd.to_datetime(arg=futureData[expObj.dateCol], format=expObj.pred_data_datetime_format)

    reqCol=[expObj.dateCol]
    for var in expObj.predVar:
        reqCol.append(var)
    futureData=futureData[reqCol] # keep only required col

    return futureData

# check Nan values in the data
def checkNaNValue(df, expObj, isHistory=True):
    for col in df:
        if df[col].isnull().sum() > 0:
            if isHistory:
                raise ValueError(
                    'Missing or Null value in the last ' + str(expObj.nLags) + ' data point for ' + str(col))
            else:
                raise ValueError('Missing or Null value in the predicted data for ' + str(col))
    return

# check Future data quality
def checkFutureData(df, expObj, lastHistDay, lastHistHour, lastHistMinute):
    # check if future data has required number of samples

    histEnd=lastHistDay.replace(hour=lastHistHour, minute=lastHistMinute)
    predStart = histEnd + pd.Timedelta(minutes=expObj.resolution)
    predEnd = histEnd + pd.Timedelta(minutes=expObj.maxPredStep * expObj.resolution)
    reqInd = list(df.index[(df[expObj.dateCol]>= predStart) & (df[expObj.dateCol] <= predEnd)])
    if len(reqInd) < expObj.maxPredStep:
        raise ValueError('Enough predicted future data is not provided')
    df.reset_index(inplace=True, drop=True)
    df=df.loc[reqInd,:]

    # check if future data has any missing data
    checkNaNValue(df=df, expObj=expObj, isHistory=False)

## Experiment Setup

In [4]:
class Experimentsetup():
    def __init__(self, config: dict):
        """
        :param config:  a dict containing all configurations necessary to prepare data for prediction models
        """
        # data set identifier
        self.dataSetName=config['dataSetName'] # data set/solar collector name

        # data range
        self.dataRange=None #possible range (max-min) of power output, it is required for computing MRE accuracy metrics

        # directories
        self.dataDir=config['dataDir'] # directory containing the data from all sources
        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.resDir = config['resDir'] # dir where to save the prediction results/accuracy

        # info on site data
        self.site_data_file = config['site_data_file'] # csv/xl file containing the historical raw data (including Ambient Temp) from site
        self.baseResolution=config['baseResolution'] # resolution (in minutes) of the raw data obtained from site
        self.site_datetime_format = config['site_datetime_format'] # format of date (in NSW local time zone) for site data
        self.dataExclusion=config['dataExclusion'] # exclude data for the date range specified. if only single date is provided, exclude data for all days forward starting that date
        self.waterSpecHeat=config['waterSpecHeat'] # this is require to compute Flow rate from raw data
        self.colAlias=config['colAlias'] # name of the columns expected in site_data_file and how to rename them later

        # information on processed/cleaned site data
        self.resolution=config['resolution'] # desired resolution (in minutes) of site data after cleaning/processing
        self.samplePerHour=int(60 / self.resolution) # samples per hour
        self.samplesPerDay = int(24 * 60 / self.resolution) # samples per day
        self.targetCol=config['targetCol'] # name of the target col
        self.dateCol=config['dateCol']  # name of the col containing timestamp
        self.inletTempCol=config['inletTempCol'] # name of the col containing inlet fluid temperature
        self.outletTempCol=config['outletTempCol'] # name of the col containing outlet fluid temperature
        self.chillerStateCol=config['chillerStateCol'] # name of the col containing the chiller's state
        self.ambTempCol=config['ambTempCol']  # name of the col containing ambient temperature
        self.flowrateCol=config['flowrateCol'] # name of the col containing flow rate
        self.deltaTempCol=config['deltaTempCol'] # name of the col containing temperature difference (outlet temp - inlet temp)

        # parameters for data cleaning and outliers detection
        if config['dataWindow']: # check data quality (e.g. missing samples) only within this window of time for each day
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
        else:
            self.windowStart=None
            self.windowEnd=None
        self.chillerOffThreshold=config['chillerOffThreshold'] # accepted highest percentage of chiller off state within dataWindow for each day
        self.percentMissingThreshold=config['percentMissingThreshold'] # accepted highest percentage of missing data within dataWindow for each day
        self.percentZeroThreshold=config['percentZeroThreshold'] # accepted highest percentage of zero valued data within dataWindow for each day
        self.minFRthreshold=config['minFRthreshold'] # min threshold for flow rate
        self.frWindowLen=config['frWindowLen'] # time (in minutes) to check the flow rate continuation for manual outliers detection
        self.maxPowerThreshold=config['maxPowerThreshold'] # maximum possible power output from the solar collector
        self.minPowerThreshold=config['minPowerThreshold'] # minimum possible power output (this is required for detecting days with only few small spikes)
        self.outliersDetMethods=config['outliersDetMethods'] # configuration parameters for outlier detection method

        # information on historical GHI/DNI data
        self.hist_GHI_file=config['hist_GHI_file'] # csv file containing historical GHI
        self.hist_DNI_file=config['hist_DNI_file'] # csv file containing historical DNI
        self.hist_GHI_DNI_resolution= config['hist_GHI_DNI_resolution']  # resolution (in minutes) of histoircal GHI/DNI data
        self.hist_GHI_DNI_dateCol= config['hist_GHI_DNI_dateCol'] # column name  containing the date in hist_GHI_file and hist_DNI_file
        self.hist_GHI_DNI_valueCol= config['hist_GHI_DNI_valueCol'] # column name  containing the GHI/DNI value in hist_GHI_file/hist_DNI_file
        self.hist_GHI_DNI_datetime_format= config[ 'hist_GHI_DNI_datetime_format'] # format of date in hist_GHI_file/hist_DNI_file
        self.hist_GHI_DNI_timezone = config['hist_GHI_DNI_timezone'] # time zone of date (either 'utc, or 'local' or None. 'local'/None means NSW timezone) in hist_GHI_file/hist_DNI_file

        # information on predicted GHI/DNI data
        self.pred_data_file = config['pred_data_file'] # csv file containing predicted GHI/DNI/Amb_temp for future (required if is_training=False)
        self.pred_data_resolution=config['pred_data_resolution'] # resolution (in minutes) of predicted weather data (GHI/DNI/Amb temp) for future
        self.pred_data_dateCol = config['pred_data_dateCol'] # column name  containing the date in pred_data_file
        self.pred_data_datetime_format = config['pred_data_datetime_format'] # format of date in pred_data_file, all date in local time zone
        self.pred_GHI_valueCol=config['pred_GHI_valueCol'] # column name containing the predicted GHI data in pred_data_file
        self.pred_DNI_valueCol=config['pred_DNI_valueCol'] # column name containing the predicted DNI data in pred_data_file
        self.pred_Amb_temp_valueCol=config['pred_Amb_temp_valueCol'] # column name containing the predicted Amb temp data in pred_data_file
        self.pred_Chiller_state_valueCol=config['pred_Chiller_state_valueCol'] # column name containing the predicted Chiller state data in pred_data_file


        # how to split and shuffle (if required) before fitting data to the prediction models
        self.dataSplitInfo=config['dataSplitInfo'] # train-validation-test split
        self.doShuffle=config['doShuffle'] # whether to shuffle the days before split
        self.randSeed=config['randSeed'] # random state for shuffling to reproduce the results
        self.trainDay=None # index of days used for training, will be set when preprocessing function is called
        self.vldDay=None # index of days used for validation, will be set when preprocessing function is called
        self.testDay=None # index of days used for testing, will be set when preprocessing function is called
        self.trainInd=None # index of starting and end data point used for training, will be set when preprocessing function is called
        self.vldInd=None # index of starting and end data point used for validation, will be set when preprocessing function is called
        self.testInd=None # index of starting and end data point used for testing, will be set when preprocessing function is called
        self.trLoc=None # index of all data points used for training, will be set when preprocessing function is called
        self.vldLoc=None # index of all data points used for validation, will be set when preprocessing function is called
        self.trVdLoc=None # index of all data points used for training+validation, will be set when preprocessing function is called
        self.testLoc=None # index of all data points used for testing, will be set when preprocessing function is called

        # parameter for data normalization
        self.scalingParam=config['scalingParam'] # how to normalize the load and other exogenous variables
        self.dataScaler=None # will be set when preprocessing function is called during model training

        # specify prediction horizon and inputs for prediction models
        self.maxPredStep=config['maxPredStep'] # setup prediction horizon, 48 means up to 24 hours since the cleaned data in 30 minutes intervals
        if not self.maxPredStep:
            self.maxPredStep=self.samplesPerDay

        self.histVar=config['inputs']['histVar'] # historical input variables
        self.predVar=config['inputs']['predVar'] # predicted weather data for future
        self.calendarVar=config['inputs']['calendarVar'] # calendar inputs (DoY: day of the year; 'ToD': time of the day)

        self.nLagDay=config['inputs']['nLagDay'] # num of previous lags (in terms of day) to use as input to the models; 1: last 48 values, 2: last 96 values
        self.nLags = config['inputs']['nLagDay'] * self.samplesPerDay # num of lagged samples as input
        self.cutoffLag = config['inputs']['nLagDay'] * self.samplesPerDay # exclude i/o exmaples for training data since lags beyond zero is not practical
        self.nChannel = len(self.histVar) + len(self.calendarVar) + len(self.predVar) + 1  # total number of input variables, +1 for the power itself
        self.nFeat = self.nLags # same as num of lagged samples as input

        # times of historical observations
        self.timeLastObservation=None

    # identify the colums that are required in the data file
    def getReqColList(self):
        """
        :return reqCol: list of string representing date column and the name of input variables (excluding calendar data)
        """
        reqCol = [self.dateCol, self.targetCol]
        reqCol.extend(self.histVar)
        reqCol.extend(self.predVar)

        return reqCol

    # check if all the required col exists in the dataframe
    def checkColumns(self, data, replacePredVar=True):
        """
        :param data: dataframe
        :param replacePredVar: boolean. True - if predicted data is not available, set historical data for same variable as the predicted data, otherwise throw exception
        :return:
        """

        # check if date col exists
        if not (self.dateCol in data):
            raise ValueError ('Required time col does not exist, please check csv file')

        if not (self.targetCol in data):
            raise ValueError('Required power output col does not exist, please check csv file')

        for col in self.histVar:
            if not (col in data):
                raise ValueError('Required historical data col does not exist, please check csv file')

        for col in self.predVar:
            if not (col in data):
                if replacePredVar==False:
                    raise ValueError('Required historical data col does not exist, please check csv file')
                else:
                    usPos = col.index('_')
                    relatedCol = col[usPos + 1:]
                    data[col] = copy.deepcopy(data[relatedCol].values)

        return

    # Convert a data vector into ndarray by rolling a window
    def getRollingDataMatrix(self, dataArray, ind):
        """
        :param dataArray: vector (1d numpy array) of data
        :param ind: list of start and end index to consider when rolling the window
        :return dataMatrix: 2d numpy array
        """

        startInd = ind[0]
        endInd = ind[1]
        nRows = endInd - startInd + 1

        dataMatrix = np.empty((nRows, self.maxPredStep))
        dataMatrix[:] = np.nan
        # timeStepVec=[]

        rowCount = -1
        for i in range(startInd, endInd + 1):
            x = i
            y = i + self.maxPredStep - 1
            y = np.clip(y, 0, endInd)
            numSampAvailable = y - x + 1
            rowCount = rowCount + 1

            dataMatrix[rowCount, 0:numSampAvailable] = dataArray[x:y + 1]
            # timeStepVec.append(timeArray[i])

        return dataMatrix  # , timeStepVec

    # find the index of the first and last samples of training, validation, and testing data
    def getSplitIndex(self, dataDF, discardLastTestDay=False):
        """
        :param dataDF: data frame containing the data from all sources
        :param discardLastTestDay: boolean. True - ingore last day of data since we don't have data beyond that day for comparison
        :return:
        """
        samplesPerDay=int(24*60/self.resolution)
        nSamples = dataDF.shape[0]
        nDays = int(nSamples / samplesPerDay)

        trainPercent = self.dataSplitInfo['train']
        nTrainDay = int(np.ceil((nDays * trainPercent) / 100))
        trainStart = 0
        trainEnd = (nTrainDay * samplesPerDay) - 1

        vldPercent = self.dataSplitInfo['vld']
        nVldDay = int(np.ceil((nDays * vldPercent) / 100))
        vldStart = trainEnd + 1
        vldEnd = ((nTrainDay + nVldDay) * samplesPerDay) - 1

        testPercent = self.dataSplitInfo['vld']
        nTestDay = nDays - nTrainDay - nVldDay
        testStart = vldEnd + 1
        testEnd = nSamples - 1

        self.trainDay = [0, nTrainDay - 1]
        self.vldDay = [nTrainDay, nTrainDay + nVldDay - 1]
        self.testDay= [nTrainDay + nVldDay, nDays - 1]

        self.trainInd = [trainStart, trainEnd]
        self.vldInd = [vldStart, vldEnd]
        self.testInd = [testStart, testEnd]

        if discardLastTestDay:
            self.testDay[1] = self.testDay[1] - 1
            self.testInd[1] = self.testInd[1] - samplesPerDay

        self.trLoc = list(range(self.trainInd[0], self.trainInd[1] + 1))
        self.vldLoc = list(range(self.vldInd[0], self.vldInd[1] + 1))

        self.trVdLoc = list(range(self.trainInd[0], self.vldInd[1] + 1))

        self.testLoc = list(range(self.testInd[0], self.testInd[1] + 1))

        return

    # prepare input-out examples for train and testing
    def prepIOExamples(self, dataDF, dataLoc, type, conv2DF=False, getTarget=False):
        """
        :param dataDF:
        :param dataLoc:
        :param type:
        :param conv2DF:
        :param getTarget:
        :return:
        """


        numRows = len(dataLoc)  # number of samples

        inputMatrix = np.empty((numRows, self.nFeat, self.nChannel))  # input data matrix
        inputMatrix[:] = np.nan

        outputMatrix = np.empty((numRows, self.maxPredStep))
        outputMatrix[:] = np.nan

        rowCount = -1
        for loc in dataLoc:
            # print(str(rowCount)+'/'+str(numRows))

            rowCount = rowCount + 1
            featVec = np.empty((self.nFeat, self.nChannel))
            featVec[:] = np.nan

            # get the lagged power and exog data
            startLag = loc - self.nLags
            endLag = loc - 1
            inputLags = np.linspace(start=startLag, stop=endLag, endpoint=True, num=self.nLags)
            inputLags = inputLags.astype(int)

            posInd = np.argwhere(inputLags >= 0)[:, 0]
            featCount = -1

            if posInd.shape[0] > 0:
                posLags = inputLags[posInd]
                curFeat = np.array([np.nan] * self.nLags)
                targetLagged = list(dataDF.loc[posLags, self.targetCol].values)
                curFeat[posInd] = targetLagged
                curFeat, _ = self.getScaledData(dataArray=curFeat, scalingParam=None, fittedScaler=self.dataScaler['target'])
                featCount = featCount + 1
                featVec[:, featCount] = curFeat

                for exogVar in self.histVar:  # lagged exog data
                    curFeat = np.array([np.nan] * self.nLags)
                    exogLagged = list(dataDF.loc[posLags, exogVar].values)
                    curFeat[posInd] = exogLagged
                    curFeat, _ = self.getScaledData(dataArray=curFeat, scalingParam=None, fittedScaler=self.dataScaler[exogVar])
                    featCount = featCount + 1
                    featVec[:, featCount] = curFeat

            else:
                curFeat = np.zeros(self.nFeat)
                curFeat[:] = np.nan
                featCount = featCount + 1
                featVec[:, featCount] = curFeat

                for exogVar in self.histVar:
                    featCount = featCount + 1
                    featVec[:, featCount] = curFeat

            # add time information
            if self.calendarVar:
                if loc > 0:
                    timeStart = dataDF.loc[loc - 1, self.dateCol]

                    for var in self.calendarVar:
                        if var == 'ToD':
                            hour = timeStart.hour
                            minute = timeStart.minute
                            dtInfo = int((hour * self.samplePerHour) + (minute / self.resolution))
                            curFeat = np.zeros(self.samplesPerDay)
                            curFeat[dtInfo] = 1

                            if curFeat.shape[0]<self.nFeat:
                                padLen=self.nFeat-curFeat.shape[0]
                                zeroVec=np.zeros(padLen)
                                curFeat=np.hstack(zeroVec, curFeat)


                        elif var == 'MoY+DoM':
                            domFeat = np.zeros(31)
                            dom = (timeStart.day) - 1
                            domFeat[dom] = 1

                            moyFeat = np.zeros(12)
                            moy = int(timeStart.month) - 1
                            moyFeat[moy] = 1

                            curFeat = np.hstack((np.zeros(5), domFeat, moyFeat))

                        elif var == 'DoY':
                            doy = timeStart.dayofyear
                            curFeat = list(np.binary_repr(doy, width=self.nFeat))
                            curFeat = np.array(curFeat, dtype=int)

                        else:
                            print('wrong time feat')
                            exit(-1)

                        featCount = featCount + 1
                        featVec[:, featCount] = curFeat
                else:
                    for var in self.calendarVar:
                        curFeat = np.zeros(self.nFeat)
                        curFeat[:] = np.nan
                        featCount = featCount + 1
                        featVec[:, featCount] = curFeat

            if self.predVar:
                targetLoc = np.linspace(start=loc, stop=loc + self.maxPredStep - 1, endpoint=True, num=self.maxPredStep)
                targetLoc = targetLoc.astype(int)
                validPredInd = np.argwhere(targetLoc <= np.nanmax(dataDF.shape[0] - 1))[:, 0]
                for var in self.predVar:
                    predFeat = np.array([np.nan] * self.maxPredStep)
                    if validPredInd.shape[0] > 0:
                        validTargetLoc = targetLoc[validPredInd]
                        futData = dataDF.loc[validTargetLoc, var].values
                        predFeat[validPredInd] = futData
                        predFeat, _ = self.getScaledData(dataArray=predFeat, scalingParam=None, fittedScaler=self.dataScaler[var])

                    featCount = featCount + 1
                    featVec[:, featCount] = predFeat

            inputMatrix[rowCount, :, :] = copy.deepcopy(featVec)

            # get the target
            if getTarget:
                targetLoc = np.linspace(start=loc, stop=loc + self.maxPredStep - 1, endpoint=True, num=self.maxPredStep)
                targetLoc = targetLoc.astype(int)
                validPredInd = np.argwhere(targetLoc <= np.nanmax(dataDF.shape[0]-1))[:, 0]
                if validPredInd.shape[0] > 0:
                    validTargetLoc = targetLoc[validPredInd]
                    targetVec = dataDF.loc[validTargetLoc, self.targetCol].values
                    targetVec, _ = self.getScaledData(dataArray=targetVec, scalingParam=None, fittedScaler=self.dataScaler['target'])
                    outputMatrix[rowCount, validPredInd] = targetVec

        # get time of last data sample
        lastObsInd=np.array(dataLoc)-1
        negInd = np.argwhere(lastObsInd<0)[:, 0]
        lastObsInd=np.clip(lastObsInd, a_min=0, a_max=max(lastObsInd))
        lastObsInd=lastObsInd.astype(int)
        timeLastObservation=dataDF.loc[lastObsInd, self.dateCol].values
        if negInd.shape[0]>0:
            negInd = list(negInd.astype(int))
            timeLastObservation[negInd]=np.datetime64('NaT')
        timeLastObservation=list(timeLastObservation)

        # clip the feature matrix
        if type.lower()=='train':
            startInd = self.cutoffLag
            inputMatrix=inputMatrix[startInd:,:,:]
            if getTarget:
                outputMatrix=outputMatrix[startInd:,:]
            timeLastObservation=timeLastObservation[startInd:]

        if conv2DF:
            inputMatrix = pd.DataFrame(inputMatrix)
            outputMatrix = pd.DataFrame(outputMatrix)


        return inputMatrix, outputMatrix, timeLastObservation

    # normalize the data in the range specified
    def getScaledData(self, dataArray, scalingParam, fittedScaler=None):
        """
        :param dataArray:
        :param scalingParam:
        :param fittedScaler:
        :return:
        """
        inputVec = copy.deepcopy(dataArray)
        inputVec = inputVec[:, np.newaxis]
        if not fittedScaler:
            dataScaler = MinMaxScaler(feature_range=(scalingParam['minValue'], scalingParam['maxValue']))
            dataScaler.fit(inputVec)
        else:
            dataScaler = fittedScaler

        scaledData = dataScaler.transform(inputVec)
        scaledData = scaledData.flatten()
        return scaledData, dataScaler

    # transform the data back to orginal scale
    def inverseScaling(self, dataArray, dataScaler):
        """
        :param dataArray:
        :param dataScaler:
        :return:
        """
        inputVec = copy.deepcopy(dataArray)
        inputVec = inputVec[:, np.newaxis]

        unScaledData = dataScaler.inverse_transform(inputVec)
        unScaledData = unScaledData.flatten()

        return unScaledData

    # instantiate the scaler for each input variable
    def getDataScaler(self, orgData, dataLoc):
        """
        :param orgData:
        :param dataLoc:
        :return:
        """
        dataScaler = dict()

        # scaler for target variable
        targetData = orgData.loc[dataLoc, self.targetCol].values
        _, targetScaler = self.getScaledData(dataArray=targetData, scalingParam=self.scalingParam, fittedScaler=None)
        dataScaler['target'] = targetScaler

        # scaler for exog var
        for var in self.histVar:
            exogTrVdData = orgData.loc[dataLoc, var].values
            _, curScaler = self.getScaledData(dataArray=exogTrVdData, scalingParam=self.scalingParam, fittedScaler=None)
            dataScaler[var] = copy.deepcopy(curScaler)

        # scaler for predicted var
        for var in self.predVar:
            predTrVdData = orgData.loc[dataLoc, var].values
            _, curScaler = self.getScaledData(dataArray=predTrVdData, scalingParam=self.scalingParam, fittedScaler=None)
            dataScaler[var] = copy.deepcopy(curScaler)

        # scaler for time var
        for var in self.calendarVar:
            if var == 'DoY':
                tempData = np.array(range(0, 366))
            elif var == 'ToD':
                tempData = np.array(range(0, self.samplesPerDay))

            else:
                print('Bad name for time variable')
                exit(-1)

            _, curScaler = self.getScaledData(dataArray=tempData, scalingParam=self.scalingParam, fittedScaler=None)
            dataScaler[var] = copy.deepcopy(curScaler)

        return dataScaler

## Forecast Model

In [5]:
class Forecastmodel():
    def __init__(self, expConfig: dict, modelConfig:dict):
        self.modelConfig=modelConfig
        if modelConfig['dataWindow']:
            self.modelConfig['dataWindow'].append(expConfig['resolution'])
        self.modelConfig['randSeed']=expConfig['randSeed'] # fix random seed for reproducibility
        self.modelConfig['modelDir']=expConfig['modelDir'] # directory where to save the trained models or their optimal parameters
        self.modelConfig['maxPredStep']=expConfig['maxPredStep']

        # instantiate an object of Experimentsetup class
        self.expObj = Experimentsetup(config=expConfig)

    def preprocess_chromasun(self, is_training, noisyData):

        # preprocess the data
        print('Data preparation is in progress........, please wait')
        orgData = datapreparation_chromasun(is_training=is_training, expObj=self.expObj, noisyData = noisyData)  # dataframe containing the clean/process data for training/testing

        return orgData
    
    def preprocess_greenland(self, is_training, noisyData):

        # preprocess the data
        print('Data preparation is in progress........, please wait')
        orgData = datapreparation_greenland(is_training=is_training, expObj=self.expObj, noisyData = noisyData)  # dataframe containing the clean/process data for training/testing

        return orgData

    def train(self, orgData):
        is_training=True
        # get the IO examples for training set
        print('Preparing IO examples for training data set')
        trainInput, trainOutput, trHistTime = self.expObj.prepIOExamples(dataDF=orgData, dataLoc=self.expObj.trLoc, type='train', conv2DF=False, getTarget=True)
        print(trainInput.shape)
        print(trainOutput.shape)
        
        
        # get the IO examples for validation set
        print('Preparing IO examples for validation data set')
        vldInput, vldOutput, vldHisTime  = self.expObj.prepIOExamples(dataDF=orgData, dataLoc=self.expObj.vldLoc, type='vld',conv2DF=False, getTarget=True)
        print(vldInput.shape)
        print(vldOutput.shape)
        
        # get the IO examples for testing set
        print('Preparing IO examples for testing data set')
        testInput, testOutput, testHistTime = self.expObj.prepIOExamples(dataDF=orgData, dataLoc=self.expObj.testLoc, type='test', conv2DF=False, getTarget=True)
        print(testInput.shape)
        print(testOutput.shape)
        
        # write target matrix (NxS where N=number of test samples, S=number of prediction steps) for test data into the disk to faciliate comparison with the predicted data later obtained from models
        actTestData=self.expObj.getRollingDataMatrix(dataArray=orgData[self.expObj.targetCol].values, ind=self.expObj.testInd)
        stepNames = list(range(1, self.expObj.maxPredStep + 1))
        stepNames = ['Step_' + str(s) for s in stepNames]
        writeDataToCSV(data=actTestData, outputFile=os.path.join(self.expObj.resDir, 'actual_data_' + self.expObj.dataSetName + '.csv'),
                       writeIndex=True, convertToDF=stepNames, indexCol=testHistTime)
        writeDataToJSON(data=actTestData, outputFile=os.path.join(self.expObj.resDir, 'actual_data_' + self.expObj.dataSetName + '.json'),
                        convertToDF=stepNames, indexCol=testHistTime, convertDateToString=True)

        # train models and save the trained models in model directory; and evaluate the trained models using testing data
        for modelName in self.modelConfig['models']:
            if modelName in ['CNN']:
                predModel = CNN(config=self.modelConfig) # instantiate the object for prediction model
            elif modelName in ['LSTM']:
                predModel= RNN(config=self.modelConfig, modelName=modelName) # instantiate the object for prediction model
            elif modelName in ["TCN"]:
                predModel = MyTCN(config=self.modelConfig)
            elif modelName in ['TDCNN']:
                predModel = TDCNN(config=self.modelConfig)
            elif modelName in ['TDCNNLSTM']:
                predModel = TDCNNLSTM(config = self.modelConfig)
            elif modelName in ['TCAN']:
                predModel = MyAttentionTCN(config = self.modelConfig)
                
            # train the model
            predModel.train(trainInput=trainInput, trainOutput=trainOutput, vldInput=vldInput, vldOutput=vldOutput, vldHisTime=vldHisTime)

            # compute the prediction for test data to evaluate the trained model
            predData = predModel.predict(inputMatrix=testInput, histObsTime=testHistTime, deNormalize=True)

            # save the predictions for test data into csv/json files in result directory
            savePredictedData(allPred=predData, dataSetName=self.expObj.dataSetName, resDir=self.expObj.resDir, maxPredStep=self.expObj.maxPredStep, histObsTime=testHistTime, is_training=is_training)

            # evaluate the prediction accuracy for test data and save the accuracy in the result directory
            accuracyDF=evaluatePrediction(allPred=predData, actData=actTestData, dataRange=self.expObj.dataRange)
            accuracyFile=os.path.join(self.expObj.resDir, 'accuracy_'+predModel.modelName+'_'+self.expObj.dataSetName+'.csv')
            writeDataToCSV(data=accuracyDF, outputFile=accuracyFile, writeIndex=False, convertToDF=None)

    def predict(self, orgData):
        is_training=False
        # prepare input data for model
        print('Preparing IO example')
        testInput, _, _ = self.expObj.prepIOExamples(dataDF=orgData, dataLoc=self.expObj.testLoc, type='test', conv2DF=False, getTarget=False)
        testHistTime=[self.expObj.timeLastObservation] # timestamp (in half-hourly interval) of last data recorded as site

        for modelName in self.modelConfig['models']:
            if modelName in ['CNN']:
                predModel = CNN(config=self.modelConfig) # instantiate the object for prediction model
            elif modelName in ['RNN']:
                predModel= RNN(config=self.modelConfig, modelName=modelName) # instantiate the object for prediction model
            elif modelName in ["TCN"]:
                pass

            predModel.load() # no train required, only load trained model from disk

            # compute the prediction
            predData = predModel.predict(inputMatrix=testInput, histObsTime=testHistTime, deNormalize=True)

            # save the predictions into csv/json files in result directory
            savePredictedData(allPred=predData, dataSetName=self.expObj.dataSetName, resDir=self.expObj.resDir, maxPredStep=self.expObj.maxPredStep, histObsTime=testHistTime, is_training=is_training)

            # prepare the dataframe containing predicted data
            timeStart=self.expObj.timeLastObservation+pd.Timedelta(minutes=self.expObj.resolution)
            timeEnd=self.expObj.timeLastObservation+ pd.Timedelta(minutes=self.expObj.resolution*self.expObj.maxPredStep)
            horizonTime = pd.date_range(start=timeStart, end=timeEnd, freq=str(self.expObj.resolution) + 'min')

            finalPrediction=pd.DataFrame()
            newCol=[np.datetime64('NaT')]*self.expObj.maxPredStep
            newCol[0]=testHistTime[0]
            finalPrediction['Time_last_data_sample']=newCol
            finalPrediction['Time_prediction_for']=horizonTime
            finalPrediction[self.expObj.targetCol]=predData[modelName].flatten()

            return finalPrediction

# MODEL CLASSES

## CNN

In [6]:
class CNN():
    def __init__(self, config: dict):
        """
        :param config: a dict containing all config necessary for the model to operate
        """
        self.modelName='CNN' # model name
        self.configParam = config['models']['CNN'] # parameters
        self.loadSavedParam=config['loadSavedParam'] # used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
        self.ensembleSize=config['ensembleSize'] # ensemble size
        self.randSeed=config['randSeed'] # fix random seed for reproducibility

        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.maxPredStep=config['maxPredStep'] # max prediction step

        if config['dataWindow']: # set the prediction outside this window as '0' regardless of the model's output.
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
            self.timeIncrement=config['dataWindow'][2]
        else:
            self.windowStart=None
            self.windowEnd=None
            self.timeIncrement=None

        self.trainedModels=None # store the trained model

    # train the CNN model
    def train(self, trainInput, trainOutput, vldInput, vldOutput, vldHisTime):
        """
        :param trainInput:
        :param trainOutput:
        :param vldInput:
        :param vldOutput:
        :param vldHisTime:
        :return:
        """

        print('Staus: Training   Model: ' + self.modelName)

        nFeat=trainInput.shape[1]
        nChannel=trainInput.shape[2]

        if self.loadSavedParam:
            paramFile = os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl')
            optParam = pickle.load(open(paramFile, 'rb'))
        else:
            modelCount = 0
            optParam = dict()
            for param in self.configParam:
                if not (param == 'searchParam'):
                    optParam[param] = copy.deepcopy(self.configParam[param][0])

            # build the initial model
            modelCount = modelCount + 1
            print(self.modelName + ' # ' + str(modelCount))

            model = self.createCNN(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'], convActFunc=optParam['convActFunc'],
                                      pooling=optParam['pooling'], batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],
                                      denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                      maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

            print(model.summary())
            
            # build subsequent models is required
            if self.configParam['searchParam']:
                # evaluate the previous model using validation data
                model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                model.fit(x=trainInput, y=trainOutput, batch_size=optParam['batchSize'], epochs=optParam['epochs'],
                             verbose=0, validation_split=optParam['vldSplit'])

                self.trainedModels=[model]

                predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                errDF.set_index('Steps', drop=True, inplace=True)
                optParam['bestValdAcc'] = errDF.loc['overall', 'RMSE']

                for param in self.configParam:
                    if param in ['searchParam', 'bestValdAcc']:
                        continue
                    convFilters = optParam['convFilters']
                    kernelSize = optParam['kernelSize']
                    convActFunc = optParam['convActFunc']
                    pooling = optParam['pooling']
                    batchNorm = optParam['batchNorm']
                    dropoutRate = optParam['dropoutRate']
                    denseNeurons = optParam['denseNeurons']
                    denseActFunc = optParam['denseActFunc']
                    batchSize = optParam['batchSize']
                    epochs = optParam['epochs']
                    vldSplit = optParam['vldSplit']

                    for i in range(1, len(self.configParam[param])):
                        curParamValue = self.configParam[param][i]
                        if param == 'convFilters':
                            convFilters = curParamValue
                        elif param == 'kernelSize':
                            kernelSize = curParamValue
                        elif param == 'convActFunc':
                            convActFunc = curParamValue
                        elif param == 'pooling':
                            pooling = curParamValue
                        elif param == 'batchNorm':
                            batchNorm = curParamValue
                        elif param == 'dropoutRate':
                            dropoutRate = curParamValue
                        elif param == 'denseNeurons':
                            denseNeurons = curParamValue
                        elif param == 'denseActFunc':
                            denseActFunc = curParamValue
                        elif param == 'batchSize':
                            batchSize = curParamValue
                        elif param == 'epochs':
                            epochs = curParamValue
                        elif param == 'vldSplit':
                            vldSplit = curParamValue

                        modelCount = modelCount + 1
                        model = self.createCNN(convFilters=convFilters, kernelSize=kernelSize, convActFunc=convActFunc,
                                          pooling=pooling, batchNorm=batchNorm, dropoutRate=dropoutRate,
                                          denseNeurons=denseNeurons, denseActFunc=denseActFunc, maxPredStep=self.maxPredStep,
                                          numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

                        model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                        model.fit(x=trainInput, y=trainOutput, batch_size=batchSize, epochs=epochs, verbose=0,
                                  validation_split=vldSplit)
                        # previous best model
                        prevBestModel=self.trainedModels

                        # set the current model as the best model
                        self.trainedModels=[model]

                        # evaluate the previous model using validation data
                        predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                        errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                        errDF.set_index('Steps', drop=True, inplace=True)
                        curMetric = errDF.loc['overall', 'RMSE']

                        if curMetric < optParam['bestValdAcc']:
                            optParam[param] = curParamValue
                            optParam['bestValdAcc'] = curMetric
                        else:
                            self.trainedModels=prevBestModel # restore the previous best model

                # save the best parameters in file
                pickle.dump(optParam,open(os.path.join(self.modelDir, 'opt_param_' + self.modelName+ '.pkl'), 'wb'))

        # train the best model using training and validation data combined
        trFeat = np.vstack((trainInput, vldInput))
        trTarget = np.vstack((trainOutput, vldOutput))

        print('Building opt ' + self.modelName)

        trainedModels = []
        for i in range(0, self.ensembleSize):
            # print('Ensembel member: ' +str(i+1)+'/'+str(ensembleSize))
            curSeed = self.randSeed + i * 10
            curModel = self.createCNN(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'],
                                 convActFunc=optParam['convActFunc'],
                                 pooling=optParam['pooling'], batchNorm=optParam['batchNorm'],
                                 dropoutRate=optParam['dropoutRate'],
                                 denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                 maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed)

            curModel.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
            curModel.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'], verbose=0,
                         validation_split=optParam['vldSplit'])

            trainedModels.append(curModel)

        self.trainedModels=trainedModels
        self.save()

        return

    # create CNN model object and configure then
    def createCNN(self, convFilters, kernelSize, convActFunc, pooling, batchNorm, dropoutRate, denseNeurons, denseActFunc,
                  maxPredStep, numFeat, numChannel, randSeed=111):
        """
        :param convFilters:
        :param kernelSize:
        :param convActFunc:
        :param pooling:
        :param batchNorm:
        :param dropoutRate:
        :param denseNeurons:
        :param denseActFunc:
        :param maxPredStep:
        :param numFeat:
        :param numChannel:
        :param randSeed:
        :return:
        """
        np.random.seed(randSeed)
        model = Sequential()
        layerCount = -1
        for nFilters in convFilters:
            layerCount = layerCount + 1

            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()
            if layerCount == 0:
                model.add(Conv1D(filters=nFilters, kernel_size=kernelSize, kernel_initializer=kernelInitalizer,
                                 bias_initializer=biasInitalizer, activation=convActFunc, padding='same',
                                 input_shape=(numFeat, numChannel)))
            else:
                model.add(Conv1D(filters=nFilters, kernel_size=kernelSize, kernel_initializer=kernelInitalizer,
                                 bias_initializer=biasInitalizer, activation=convActFunc, padding='same'))

            if pooling == 'max':
                model.add(MaxPooling1D(pool_size=2, padding='same'))
            elif pooling == 'avg':
                model.add(AveragePooling1D(pool_size=2, padding='same'))
            if batchNorm:
                model.add(BatchNormalization())
            if dropoutRate:
                model.add(Dropout(dropoutRate))

        # add flatten layer
        model.add(Flatten())

        # add dense layers as required
        for neurons in denseNeurons:
            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()
            model.add(Dense(neurons, kernel_initializer=kernelInitalizer, bias_initializer=biasInitalizer,
                            activation=denseActFunc))

        randSeed = randSeed + 1
        kernelInitalizer = glorot_uniform(seed=randSeed)

        model.add(Dense(maxPredStep, kernel_initializer=kernelInitalizer, bias_initializer='zeros',
                        activation='linear'))  # output layer

        return model

    # make prediction
    def predict(self, inputMatrix, histObsTime=None, deNormalize=False):
        """
        :param inputMatrix:
        :param predStartTime:
        :param deNormalize:
        :return:
        """
        # print('Status: Testing')
        if deNormalize:
            scalerFile=os.path.join(self.modelDir, 'scaler_target.pkl')
            dataScaler=pickle.load(open(scalerFile, 'rb'))

        numTestSample = inputMatrix.shape[0]
        allPred={self.modelName: np.empty((numTestSample, self.maxPredStep))}
        allPred[self.modelName][:] = np.nan

        if self.windowStart:
            windowStart = pd.to_datetime(arg=self.windowStart, format="%H:%M")
            windowStart = windowStart.replace(year=2000, month=1, day=1)
            windowEnd = pd.to_datetime(arg=self.windowEnd, format="%H:%M")
            windowEnd = windowEnd.replace(year=2000, month=1, day=1)

        for rowCount in range(numTestSample):
            feat=np.zeros(shape=(1, inputMatrix.shape[1], inputMatrix.shape[2]))
            feat[0,:,:] = inputMatrix[rowCount,:]
            predData = np.zeros((len(self.trainedModels), self.maxPredStep))

            for i in range(0, len(self.trainedModels)):
                curPred = self.trainedModels[i].predict(feat)
                curPred = curPred.flatten()

                # de-normalize the prediction if required
                if deNormalize:
                    curPred = curPred[:, np.newaxis]
                    curPred = dataScaler.inverse_transform(curPred)
                    curPred = curPred.flatten()

                curPred = np.clip(curPred, 0, None)
                predData[i, :] = curPred

            predData = np.nanmean(predData, axis=0)
            allPred[self.modelName][rowCount, :] = predData

            # manually set the predicted data as zero if prediction time is outside the specified window
            if self.windowStart:
                timeStart = histObsTime[rowCount]+pd.Timedelta(minutes=self.timeIncrement)
                timeEnd = timeStart + pd.Timedelta(minutes=self.timeIncrement * (self.maxPredStep - 1))
                timeRange = pd.Series(pd.date_range(start=timeStart, end=timeEnd, freq=str(self.timeIncrement) + 'min'))
                timeRange = timeRange.apply(lambda x: x.replace(year=2000, month=1, day=1))

                outsideWindowInd = list(timeRange.index[(timeRange < windowStart) | (timeRange > windowEnd)])
                allPred[self.modelName][rowCount, outsideWindowInd] = 0

        return allPred

    # load pre-trained model from disk
    def load(self):
        """
        :return:
        """
        from keras.models import load_model
        trainedModels=[]
        for i in range(self.ensembleSize):
            fileName=os.path.join(self.modelDir, self.modelName+'_'+str(i)+'.h5')
            if not os.path.isfile(fileName):
                raise FileNotFoundError('Model file does not exist')

            model=load_model(filepath=fileName)
            trainedModels.append(model)
            self.trainedModels=trainedModels

        return

    # save trained model to disk
    def save(self):
        """
        :return:
        """
        for i in range(self.ensembleSize):
            fileName = os.path.join(self.modelDir, self.modelName + '_' + str(i) + '.h5')
            self.trainedModels[i].save(fileName)

        return

## RNN/LSTM/GRU

In [ ]:
class RNN():
    def __init__(self, config: dict, modelName='LSTM'):
        """
        :param config: a dict containing all config necessary for the model to operate
        """
        self.modelName=modelName # model name
        self.configParam = config['models'][self.modelName]  # parameters
        self.loadSavedParam=config['loadSavedParam'] # used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
        self.ensembleSize=config['ensembleSize'] # ensemble size
        self.randSeed=config['randSeed'] # fix random seed for reproducibility

        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.maxPredStep=config['maxPredStep'] # max prediction step

        if config['dataWindow']: # set the prediction outside this window as '0' regardless of the model's output.
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
            self.timeIncrement=config['dataWindow'][2]
        else:
            self.windowStart=None
            self.windowEnd=None
            self.timeIncrement=None

        self.trainedModels=None

    # train the LSTM/GRU model
    def train(self, trainInput, trainOutput, vldInput, vldOutput, vldHisTime):
        """
        :param trainInput:
        :param trainOutput:
        :param vldInput:
        :param vldOutput:
        :param vldHisTime:
        :return:
        """
        print('Staus: Training   Model: ' + self.modelName)
        nFeat=trainInput.shape[1]
        nChannel=trainInput.shape[2]

        if self.loadSavedParam:
            paramFile = os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl')
            optParam = pickle.load(open(paramFile, 'rb'))

        else:
            trFeat=trainInput
            trTarget=trainOutput
            if self.modelName.lower() == 'lstm_endec':
                trTarget=trTarget.reshape((trTarget.shape[0], trTarget.shape[1], 1))

            modelCount = 0
            optParam = dict()
            for param in self.configParam:
                if not (param == 'searchParam'):
                    optParam[param] = copy.deepcopy(self.configParam[param][0])

            # build the initial model
            modelCount = modelCount + 1
            print(self.modelName + ' # ' + str(modelCount))

            if self.modelName in ['LSTM', 'GRU']:
                model = self.createLSTM(lstmBlocks=optParam['lstmBlocks'], lstmActFunc=optParam['lstmActFunc'],
                                           recActFunc=optParam['recActFunc'], batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],
                                           recDropoutRate=optParam['recDropoutRate'], denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                           maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed, modelName=self.modelName)
                print(model.summary())
                
            else:
                model = self.createLSTMendec(lstmBlocks=optParam['lstmBlocks'], lstmActFunc=optParam['lstmActFunc'], recActFunc=optParam['recActFunc'],
                                                batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],recDropoutRate=optParam['recDropoutRate'],
                                                denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                                maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

            # build subsequent models is required
            if self.configParam['searchParam']:

                # evaluate the previous model using validation data
                model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                model.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'],
                             verbose=0,validation_split=optParam['vldSplit'])

                self.trainedModels = [model] # set the current model as the best model

                predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                errDF.set_index('Steps', drop=True, inplace=True)
                optParam['bestValdAcc'] = errDF.loc['overall', 'RMSE']

                for param in self.configParam:
                    if param in ['searchParam', 'bestValdAcc']:
                        continue
                    lstmBlocks = optParam['lstmBlocks']
                    lstmActFunc = optParam['lstmActFunc']
                    recActFunc = optParam['recActFunc']
                    batchNorm = optParam['batchNorm']
                    dropoutRate = optParam['dropoutRate']
                    recDropoutRate = optParam['recDropoutRate']
                    denseNeurons = optParam['denseNeurons']
                    denseActFunc = optParam['denseActFunc']
                    batchSize = optParam['batchSize']
                    epochs = optParam['epochs']
                    vldSplit = optParam['vldSplit']

                    for i in range(1, len(self.configParam[param])):
                        curParamValue = self.configParam[param][i]
                        if param == 'lstmBlocks':
                            lstmBlocks = curParamValue
                        elif param == 'lstmActFunc':
                            lstmActFunc = curParamValue
                        elif param == 'recActFunc':
                            recActFunc = curParamValue
                        elif param == 'batchNorm':
                            batchNorm = curParamValue
                        elif param == 'dropoutRate':
                            dropoutRate = curParamValue
                        elif param == 'recDropoutRate':
                            recDropoutRate = curParamValue
                        elif param == 'denseNeurons':
                            denseNeurons = curParamValue
                        elif param == 'denseActFunc':
                            denseActFunc = curParamValue
                        elif param == 'batchSize':
                            batchSize = curParamValue
                        elif param == 'epochs':
                            epochs = curParamValue
                        elif param == 'vldSplit':
                            vldSplit = curParamValue

                        modelCount = modelCount + 1
                        # print(modelName + ' # ' + str(modelCount))

                        if self.modelName in ['LSTM', 'GRU']:
                            model = self.createLSTM(lstmBlocks=lstmBlocks, lstmActFunc=lstmActFunc, recActFunc=recActFunc, batchNorm=batchNorm,
                                                    dropoutRate=dropoutRate, recDropoutRate=recDropoutRate, denseNeurons=denseNeurons, denseActFunc=denseActFunc,
                                                    maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed, modelName=self.modelName)
                        else:
                            model = self.createLSTMendec(lstmBlocks=lstmBlocks, lstmActFunc=lstmActFunc, recActFunc=recActFunc, batchNorm=batchNorm,
                                                         dropoutRate=dropoutRate, recDropoutRate=recDropoutRate, denseNeurons=denseNeurons, denseActFunc=denseActFunc,
                                                         maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

                        model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                        model.fit(x=trFeat, y=trTarget, batch_size=batchSize, epochs=epochs, verbose=0, validation_split=vldSplit)

                        # previous best model
                        prevBestModel=self.trainedModels

                        # set the current model as the best model
                        self.trainedModels=[model]

                        # evaluate the current model using validation data
                        predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)
                        errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                        errDF.set_index('Steps', drop=True, inplace=True)
                        curMetric = errDF.loc['overall', 'RMSE']

                        if curMetric < optParam['bestValdAcc']:
                            optParam[param] = curParamValue
                            optParam['bestValdAcc'] = curMetric
                        else:
                            self.trainedModels=prevBestModel # restore the previous best model

                # save the best parameters in file
                pickle.dump(optParam,open(os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl'), 'wb'))

        # train the best model using training and validation data combined
        trFeat = np.vstack((trainInput, vldInput))
        trTarget = np.vstack((trainOutput, vldOutput))

        if self.modelName.lower() == 'lstm_endec':
            trTarget = trTarget.reshape((trTarget.shape[0], trTarget.shape[1], 1))

        print('Building opt ' + self.modelName)

        trainedModels = []
        for i in range(0, self.ensembleSize):
            curSeed = self.randSeed + i * 10
            if self.modelName in ['LSTM', 'GRU']:
                curModel = self.createLSTM(lstmBlocks=optParam['lstmBlocks'], lstmActFunc=optParam['lstmActFunc'], recActFunc=optParam['recActFunc'],
                                           batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'], recDropoutRate=optParam['recDropoutRate'],
                                           denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                           maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed, modelName=self.modelName)
            else:
                curModel = self.createLSTMendec(lstmBlocks=optParam['lstmBlocks'], lstmActFunc=optParam['lstmActFunc'],
                                           recActFunc=optParam['recActFunc'], batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],
                                           recDropoutRate=optParam['recDropoutRate'], denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                           maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed)

            curModel.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
            curModel.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'], verbose=0, validation_split=optParam['vldSplit'])
            trainedModels.append(curModel)

        self.trainedModels=trainedModels
        self.save()

        return

    # create LSTM/GRU model object and configure then
    def createLSTM(self, lstmBlocks, lstmActFunc, recActFunc, batchNorm, dropoutRate, recDropoutRate, denseNeurons,
                   denseActFunc, maxPredStep, numFeat, numChannel, randSeed=111, modelName='LSTM'):
        """
        :param lstmBlocks:
        :param lstmActFunc:
        :param recActFunc:
        :param batchNorm:
        :param dropoutRate:
        :param recDropoutRate:
        :param denseNeurons:
        :param denseActFunc:
        :param maxPredStep:
        :param numFeat:
        :param numChannel:
        :param randSeed:
        :param modelName:
        :return:
        """

        np.random.seed(randSeed)
        model = Sequential()
        layerCount = -1
        for nBlocks in lstmBlocks:
            layerCount = layerCount + 1

            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            recInitalizer = Orthogonal(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()

            if layerCount < (len(lstmBlocks) - 1):
                retSeq = True
            else:
                retSeq = False

            if layerCount == 0:
                if modelName == 'LSTM':
                    model.add(LSTM(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                                   kernel_initializer=kernelInitalizer, recurrent_initializer=recInitalizer,
                                   bias_initializer=biasInitalizer, dropout=dropoutRate,
                                   recurrent_dropout=recDropoutRate, return_sequences=retSeq,
                                   input_shape=(numFeat, numChannel)))
                else:
                    model.add(GRU(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                                  kernel_initializer=kernelInitalizer, recurrent_initializer=recInitalizer,
                                  bias_initializer=biasInitalizer, dropout=dropoutRate,
                                  recurrent_dropout=recDropoutRate, return_sequences=retSeq,
                                  input_shape=(numFeat, numChannel)))

            else:
                if modelName == 'LSTM':
                    model.add(LSTM(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                                   kernel_initializer=kernelInitalizer,
                                   recurrent_initializer=recInitalizer, bias_initializer=biasInitalizer,
                                   dropout=dropoutRate, recurrent_dropout=recDropoutRate, return_sequences=retSeq))
                else:
                    model.add(GRU(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                                  kernel_initializer=kernelInitalizer,
                                  recurrent_initializer=recInitalizer, bias_initializer=biasInitalizer,
                                  dropout=dropoutRate, recurrent_dropout=recDropoutRate, return_sequences=retSeq))

            if batchNorm:
                model.add(BatchNormalization())

        # add dense layers as required
        for neurons in denseNeurons:
            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()
            model.add(Dense(neurons, kernel_initializer=kernelInitalizer, bias_initializer=biasInitalizer,
                            activation=denseActFunc))

        randSeed = randSeed + 1
        kernelInitalizer = glorot_uniform(seed=randSeed)

        model.add(Dense(maxPredStep, kernel_initializer=kernelInitalizer, bias_initializer='zeros',
                        activation='linear'))  # output layer

        return model

    # create LSTM encoder decoder model object and configure then
    def createLSTMendec(self, lstmBlocks, lstmActFunc, recActFunc, batchNorm, dropoutRate, recDropoutRate, denseNeurons,
                        denseActFunc, maxPredStep, numFeat, numChannel, randSeed=111):
        """
        :param lstmBlocks:
        :param lstmActFunc:
        :param recActFunc:
        :param batchNorm:
        :param dropoutRate:
        :param recDropoutRate:
        :param denseNeurons:
        :param denseActFunc:
        :param maxPredStep:
        :param numFeat:
        :param numChannel:
        :param randSeed:
        :return:
        """

        np.random.seed(randSeed)
        model = Sequential()
        layerCount = -1
        for nBlocks in lstmBlocks:
            layerCount = layerCount + 1

            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            recInitalizer = Orthogonal(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()

            if layerCount < (len(lstmBlocks) - 1):
                retSeq = True
            else:
                retSeq = False

            if layerCount == 0:
                model.add(LSTM(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                               kernel_initializer=kernelInitalizer,
                               recurrent_initializer=recInitalizer, bias_initializer=biasInitalizer,
                               dropout=dropoutRate, recurrent_dropout=recDropoutRate,
                               return_sequences=retSeq, input_shape=(numFeat, numChannel)))
            else:
                model.add(LSTM(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                               kernel_initializer=kernelInitalizer,
                               recurrent_initializer=recInitalizer, bias_initializer=biasInitalizer,
                               dropout=dropoutRate, recurrent_dropout=recDropoutRate,
                               return_sequences=retSeq))

        # connection with decoder
        model.add(RepeatVector(maxPredStep))

        # decoder
        layerCount = -1
        retSeq = True  # for decoder return seq should be true always
        for nBlocks in lstmBlocks:
            layerCount = layerCount + 1

            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            recInitalizer = Orthogonal(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()

            model.add(LSTM(units=nBlocks, activation=lstmActFunc, recurrent_activation=recActFunc,
                           kernel_initializer=kernelInitalizer,
                           recurrent_initializer=recInitalizer, bias_initializer=biasInitalizer, dropout=dropoutRate,
                           recurrent_dropout=recDropoutRate,
                           return_sequences=retSeq))

        # add dense layers as required
        for neurons in denseNeurons:
            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()

            model.add(TimeDistributed(
                Dense(neurons, kernel_initializer=kernelInitalizer, bias_initializer=biasInitalizer,
                      activation=denseActFunc)))

        randSeed = randSeed + 1
        kernelInitalizer = glorot_uniform(seed=randSeed)
        biasInitalizer = Zeros()
        model.add(TimeDistributed(
            Dense(1, kernel_initializer=kernelInitalizer, bias_initializer=biasInitalizer, activation='linear')))

        return model

    # make prediction
    def predict(self, inputMatrix, histObsTime=None, deNormalize=False):
        """
        :param inputMatrix:
        :param predStartTime:
        :param deNormalize:
        :return:
        """
        # print('Status: Testing')
        if deNormalize:
            scalerFile=os.path.join(self.modelDir, 'scaler_target.pkl')
            dataScaler=pickle.load(open(scalerFile, 'rb'))

        numTestSample = inputMatrix.shape[0]
        allPred={self.modelName: np.empty((numTestSample, self.maxPredStep))}
        allPred[self.modelName][:] = np.nan

        if self.windowStart:
            windowStart = pd.to_datetime(arg=self.windowStart, format="%H:%M")
            windowStart = windowStart.replace(year=2000, month=1, day=1)
            windowEnd = pd.to_datetime(arg=self.windowEnd, format="%H:%M")
            windowEnd = windowEnd.replace(year=2000, month=1, day=1)

        for rowCount in range(numTestSample):
            feat=np.zeros(shape=(1, inputMatrix.shape[1], inputMatrix.shape[2]))
            feat[0,:,:] = inputMatrix[rowCount,:]
            predData = np.zeros((len(self.trainedModels), self.maxPredStep))

            for i in range(0, len(self.trainedModels)):
                curPred = self.trainedModels[i].predict(feat)
                curPred = curPred.flatten()

                # de-normalize the prediction if required
                if deNormalize:
                    curPred = curPred[:, np.newaxis]
                    curPred = dataScaler.inverse_transform(curPred)
                    curPred = curPred.flatten()

                curPred = np.clip(curPred, 0, None)
                predData[i, :] = curPred

            predData = np.nanmean(predData, axis=0)
            allPred[self.modelName][rowCount, :] = predData

            # manually set the predicted data as zero if prediction time is outside the specified window
            if self.windowStart:
                timeStart = histObsTime[rowCount]+pd.Timedelta(minutes=self.timeIncrement)
                timeEnd = timeStart + pd.Timedelta(minutes=self.timeIncrement * (self.maxPredStep - 1))
                timeRange = pd.Series(pd.date_range(start=timeStart, end=timeEnd, freq=str(self.timeIncrement) + 'min'))
                timeRange = timeRange.apply(lambda x: x.replace(year=2000, month=1, day=1))

                outsideWindowInd = list(timeRange.index[(timeRange < windowStart) | (timeRange > windowEnd)])
                allPred[self.modelName][rowCount, outsideWindowInd] = 0

        return allPred

    # load pre-trained model from disk
    def load(self):
        """
        :return:
        """
        from keras.models import load_model
        trainedModels=[]
        for i in range(self.ensembleSize):
            fileName=os.path.join(self.modelDir, self.modelName+'_'+str(i)+'.h5')
            if not os.path.isfile(fileName):
                raise FileNotFoundError('Model file does not exist')

            model=load_model(filepath=fileName)
            trainedModels.append(model)
            self.trainedModels=trainedModels

        return

    # save trained model to disk
    def save(self):
        """
        :return:
        """
        for i in range(self.ensembleSize):
            fileName = os.path.join(self.modelDir, self.modelName + '_' + str(i) + '.h5')
            self.trainedModels[i].save(fileName)

        return

## TCN

In [ ]:
class MyTCN():
    def __init__(self, config: dict):
        """
        :param config: a dict containing all config necessary for the model to operate
        """
        self.modelName='TCN' # model name
        self.configParam = config['models']['TCN'] # parameters
        self.loadSavedParam=config['loadSavedParam'] # used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
        self.ensembleSize=config['ensembleSize'] # ensemble size
        self.randSeed=config['randSeed'] # fix random seed for reproducibility

        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.maxPredStep=config['maxPredStep'] # max prediction step

        if config['dataWindow']: # set the prediction outside this window as '0' regardless of the model's output.
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
            self.timeIncrement=config['dataWindow'][2]
        else:
            self.windowStart=None
            self.windowEnd=None
            self.timeIncrement=None

        self.trainedModels=None    
        
    def train(self, trainInput, trainOutput, vldInput, vldOutput, vldHisTime):
        print('Status: Training Model: ' + self.modelName)
        nFeat=trainInput.shape[1]
        nChannel=trainInput.shape[2]

        if self.loadSavedParam:
            paramFile = os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl')
            optParam = pickle.load(open(paramFile, 'rb'))
        else:
            modelCount = 0
            optParam = dict()
            for param in self.configParam:
                if not (param == 'searchParam'):
                    optParam[param] = copy.deepcopy(self.configParam[param][0])

            # build the initial model
            modelCount = modelCount + 1
            print(self.modelName + ' # ' + str(modelCount))
            
            model = self.createTCN(nbFilters=optParam['nbFilters'], kernelSize=optParam['kernelSize'], 
                                dilations=optParam['dilations'], kernel_initialiser=optParam['kernel_initialiser'], 
                                batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'], 
                                denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)
            
            print(model.summary())
            
            # build subsequent models is required
            if self.configParam['searchParam']:
                # evaluate the previous model using validation data
                model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                model.fit(x=trainInput, y=trainOutput, batch_size=optParam['batchSize'], epochs=optParam['epochs'],
                             verbose=0, validation_split=optParam['vldSplit'])

                self.trainedModels=[model]

                predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                errDF.set_index('Steps', drop=True, inplace=True)
                optParam['bestValdAcc'] = errDF.loc['overall', 'RMSE']

                for param in self.configParam:
                    if param in ['searchParam', 'bestValdAcc']:
                        continue
                    nbFilters = optParam['nbFilters']
                    kernelSize = optParam['kernelSize']
                    dilations = optParam['dilations']
                    kernel_initialiser = optParam['kernel_initialiser']
                    batchNorm = optParam['batchNorm']
                    dropoutRate = optParam['dropoutRate']
                    denseNeurons = optParam['denseNeurons']
                    denseActFunc = optParam['denseActFunc']
                    batchSize = optParam['batchSize']
                    epochs = optParam['epochs']
                    vldSplit = optParam['vldSplit']

                    for i in range(1, len(self.configParam[param])):
                        curParamValue = self.configParam[param][i]
                        if param == 'nbFilters':
                            nbFilters = curParamValue
                        elif param == 'kernelSize':
                            kernelSize = curParamValue
                        elif param == 'dilations':
                            dilations = curParamValue
                        elif param == 'kernel_initialiser':
                            kernel_initialiser = curParamValue
                        elif param == 'batchNorm':
                            batchNorm = curParamValue
                        elif param == 'dropoutRate':
                            dropoutRate = curParamValue
                        elif param == 'denseNeurons':
                            denseNeurons = curParamValue
                        elif param == 'denseActFunc':
                            denseActFunc = curParamValue
                        elif param == 'batchSize':
                            batchSize = curParamValue
                        elif param == 'epochs':
                            epochs = curParamValue
                        elif param == 'vldSplit':
                            vldSplit = curParamValue

                        modelCount = modelCount + 1
                        model = self.createTCN(nbFilters=nbFilters, kernelSize=kernelSize, 
                                        dilations=dilations, kernel_initialiser=kernel_initialiser, 
                                        batchNorm=batchNorm, dropoutRate=dropoutRate, 
                                        denseNeurons=denseNeurons, denseActFunc=denseActFunc,
                                        maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

                        model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                        model.fit(x=trainInput, y=trainOutput, batch_size=batchSize, epochs=epochs, verbose=0,
                                  validation_split=vldSplit)
                        # previous best model
                        prevBestModel=self.trainedModels

                        # set the current model as the best model
                        self.trainedModels=[model]

                        # evaluate the previous model using validation data
                        predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                        errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                        errDF.set_index('Steps', drop=True, inplace=True)
                        curMetric = errDF.loc['overall', 'RMSE']

                        if curMetric < optParam['bestValdAcc']:
                            optParam[param] = curParamValue
                            optParam['bestValdAcc'] = curMetric
                        else:
                            self.trainedModels=prevBestModel # restore the previous best model

                # save the best parameters in file
                pickle.dump(optParam,open(os.path.join(self.modelDir, 'opt_param_' + self.modelName+ '.pkl'), 'wb'))
            
            # train the best model using training and validation data combined
            trFeat = np.vstack((trainInput, vldInput))
            trTarget = np.vstack((trainOutput, vldOutput))

            print('Building opt ' + self.modelName)

            trainedModels = []
            for i in range(0, self.ensembleSize):
                curSeed = self.randSeed + i * 10
                
                curModel = self.createTCN(nbFilters=optParam['nbFilters'], kernelSize=optParam['kernelSize'], 
                                        dilations=optParam['dilations'], kernel_initialiser=optParam['kernel_initialiser'], 
                                        batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'], 
                                        denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                        maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed)
                
                curModel.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                curModel.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'], verbose=0,
                             validation_split=optParam['vldSplit'])

                trainedModels.append(curModel)

            self.trainedModels=trainedModels
            self.save()

            return
    
    def createTCN(self, denseNeurons, denseActFunc, maxPredStep, numFeat, numChannel, nbFilters, 
                  kernelSize, dilations, batchNorm, dropoutRate, kernel_initialiser = "he_normal", randSeed = 111):
        
        np.random.seed(randSeed)
        model = Sequential()
        layerCount = -1
        
        for nb_filter in nbFilters:
            layerCount = layerCount + 1
            
            if layerCount < (len(nbFilters) - 1):
                retSeq = True
            else:
                retSeq = False
            
            if layerCount == 0:
                model.add(TCN(nb_filters = nb_filter, kernel_size = kernelSize, dilations = dilations, 
                              dropout_rate = dropoutRate, kernel_initializer = kernel_initialiser, 
                              use_batch_norm = batchNorm, return_sequences = retSeq, input_shape=(numFeat, numChannel)))
            else:
                model.add(TCN(nb_filters = nb_filter, kernel_size = kernelSize, dilations = dilations, 
                              dropout_rate = dropoutRate, kernel_initializer = kernel_initialiser, 
                              use_batch_norm = batchNorm, return_sequences = retSeq))
                
        # add dense layers as required
        for neurons in denseNeurons:
            randSeed = randSeed + 1
            kernelInitalizer = glorot_uniform(seed=randSeed)

            randSeed = randSeed + 1
            biasInitalizer = Zeros()
            model.add(Dense(neurons, kernel_initializer=kernelInitalizer, bias_initializer=biasInitalizer,
                            activation=denseActFunc))

        randSeed = randSeed + 1
        kernelInitalizer = glorot_uniform(seed=randSeed)

        model.add(Dense(maxPredStep, kernel_initializer=kernelInitalizer, bias_initializer='zeros',
                        activation='linear'))  # output layer

        return model
    
    # make prediction
    def predict(self, inputMatrix, histObsTime=None, deNormalize=False):
        """
        :param inputMatrix:
        :param predStartTime:
        :param deNormalize:
        :return:
        """
        
        # print('Status: Testing')
        if deNormalize:
            scalerFile=os.path.join(self.modelDir, 'scaler_target.pkl')
            dataScaler=pickle.load(open(scalerFile, 'rb'))

        numTestSample = inputMatrix.shape[0]
        allPred={self.modelName: np.empty((numTestSample, self.maxPredStep))}
        allPred[self.modelName][:] = np.nan

        if self.windowStart:
            windowStart = pd.to_datetime(arg=self.windowStart, format="%H:%M")
            windowStart = windowStart.replace(year=2000, month=1, day=1)
            windowEnd = pd.to_datetime(arg=self.windowEnd, format="%H:%M")
            windowEnd = windowEnd.replace(year=2000, month=1, day=1)

        for rowCount in range(numTestSample):
            feat=np.zeros(shape=(1, inputMatrix.shape[1], inputMatrix.shape[2]))
            feat[0,:,:] = inputMatrix[rowCount,:]
            predData = np.zeros((len(self.trainedModels), self.maxPredStep))

            for i in range(0, len(self.trainedModels)):
                curPred = self.trainedModels[i].predict(feat)
                curPred = curPred.flatten()

                # de-normalize the prediction if required
                if deNormalize:
                    curPred = curPred[:, np.newaxis]
                    curPred = dataScaler.inverse_transform(curPred)
                    curPred = curPred.flatten()

                curPred = np.clip(curPred, 0, None)
                predData[i, :] = curPred

            predData = np.nanmean(predData, axis=0)
            allPred[self.modelName][rowCount, :] = predData

            # manually set the predicted data as zero if prediction time is outside the specified window
            if self.windowStart:
                timeStart = histObsTime[rowCount]+pd.Timedelta(minutes=self.timeIncrement)
                timeEnd = timeStart + pd.Timedelta(minutes=self.timeIncrement * (self.maxPredStep - 1))
                timeRange = pd.Series(pd.date_range(start=timeStart, end=timeEnd, freq=str(self.timeIncrement) + 'min'))
                timeRange = timeRange.apply(lambda x: x.replace(year=2000, month=1, day=1))

                outsideWindowInd = list(timeRange.index[(timeRange < windowStart) | (timeRange > windowEnd)])
                allPred[self.modelName][rowCount, outsideWindowInd] = 0

        return allPred

    # load pre-trained model from disk
    def load(self):
        """
        :return:
        """
        from keras.models import load_model
        trainedModels=[]
        for i in range(self.ensembleSize):
            fileName=os.path.join(self.modelDir, self.modelName+'_'+str(i)+'.h5')
            if not os.path.isfile(fileName):
                raise FileNotFoundError('Model file does not exist')

            model=load_model(filepath=fileName)
            trainedModels.append(model)
            self.trainedModels=trainedModels

        return

    # save trained model to disk
    def save(self):
        """
        :return:
        """
        for i in range(self.ensembleSize):
            fileName = os.path.join(self.modelDir, self.modelName + '_' + str(i) + '.h5')
            self.trainedModels[i].save(fileName)

        return    

## Time Distributed CNN

In [ ]:
class TDCNN:
    def __init__(self, config: dict):
        """
        :param config: a dict containing all config necessary for the model to operate
        """
        self.modelName='TDCNN' # model name
        self.configParam = config['models']['TDCNN'] # parameters
        self.loadSavedParam=config['loadSavedParam'] # used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
        self.ensembleSize=config['ensembleSize'] # ensemble size
        self.randSeed=config['randSeed'] # fix random seed for reproducibility

        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.maxPredStep=config['maxPredStep'] # max prediction step

        if config['dataWindow']: # set the prediction outside this window as '0' regardless of the model's output.
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
            self.timeIncrement=config['dataWindow'][2]
        else:
            self.windowStart=None
            self.windowEnd=None
            self.timeIncrement=None

        self.trainedModels=None
        
    def train(self, trainInput, trainOutput, vldInput, vldOutput, vldHisTime):
        """
        :param trainInput:
        :param trainOutput:
        :param vldInput:
        :param vldOutput:
        :param vldHisTime:
        :return:
        """

        print('Staus: Training   Model: ' + self.modelName)

        nFeat=trainInput.shape[1]
        nChannel=trainInput.shape[2]

        if self.loadSavedParam:
            paramFile = os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl')
            optParam = pickle.load(open(paramFile, 'rb'))
        else:
            modelCount = 0
            optParam = dict()
            for param in self.configParam:
                if not (param == 'searchParam'):
                    optParam[param] = copy.deepcopy(self.configParam[param][0])

            # build the initial model
            modelCount = modelCount + 1
            print(self.modelName + ' # ' + str(modelCount))

            model = self.createTDCNN(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'], convActFunc=optParam['convActFunc'],
                                      pooling=optParam['pooling'], batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],
                                      denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                      maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

            print(model.summary())
            
            # build subsequent models is required
            if self.configParam['searchParam']:
                # evaluate the previous model using validation data
                model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                model.fit(x=trainInput, y=trainOutput, batch_size=optParam['batchSize'], epochs=optParam['epochs'],
                             verbose=0)

                self.trainedModels=[model]

                predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                errDF.set_index('Steps', drop=True, inplace=True)
                optParam['bestValdAcc'] = errDF.loc['overall', 'RMSE']

                for param in self.configParam:
                    if param in ['searchParam', 'bestValdAcc']:
                        continue
                    convFilters = optParam['convFilters']
                    kernelSize = optParam['kernelSize']
                    convActFunc = optParam['convActFunc']
                    pooling = optParam['pooling']
                    batchNorm = optParam['batchNorm']
                    dropoutRate = optParam['dropoutRate']
                    denseNeurons = optParam['denseNeurons']
                    denseActFunc = optParam['denseActFunc']
                    batchSize = optParam['batchSize']
                    epochs = optParam['epochs']
                    vldSplit = optParam['vldSplit']

                    for i in range(1, len(self.configParam[param])):
                        curParamValue = self.configParam[param][i]
                        if param == 'convFilters':
                            convFilters = curParamValue
                        elif param == 'kernelSize':
                            kernelSize = curParamValue
                        elif param == 'convActFunc':
                            convActFunc = curParamValue
                        elif param == 'pooling':
                            pooling = curParamValue
                        elif param == 'batchNorm':
                            batchNorm = curParamValue
                        elif param == 'dropoutRate':
                            dropoutRate = curParamValue
                        elif param == 'denseNeurons':
                            denseNeurons = curParamValue
                        elif param == 'denseActFunc':
                            denseActFunc = curParamValue
                        elif param == 'batchSize':
                            batchSize = curParamValue
                        elif param == 'epochs':
                            epochs = curParamValue
                        elif param == 'vldSplit':
                            vldSplit = curParamValue

                        modelCount = modelCount + 1
                        model = self.createTDCNN(convFilters=convFilters, kernelSize=kernelSize, convActFunc=convActFunc,
                                          pooling=pooling, batchNorm=batchNorm, dropoutRate=dropoutRate,
                                          denseNeurons=denseNeurons, denseActFunc=denseActFunc, maxPredStep=self.maxPredStep,
                                          numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

                        model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                        model.fit(x=trainInput, y=trainOutput, batch_size=batchSize, epochs=epochs, verbose=0)
                        # previous best model
                        prevBestModel=self.trainedModels

                        # set the current model as the best model
                        self.trainedModels=[model]

                        # evaluate the previous model using validation data
                        predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                        errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                        errDF.set_index('Steps', drop=True, inplace=True)
                        curMetric = errDF.loc['overall', 'RMSE']

                        if curMetric < optParam['bestValdAcc']:
                            optParam[param] = curParamValue
                            optParam['bestValdAcc'] = curMetric
                        else:
                            self.trainedModels=prevBestModel # restore the previous best model

                # save the best parameters in file
                pickle.dump(optParam,open(os.path.join(self.modelDir, 'opt_param_' + self.modelName+ '.pkl'), 'wb'))

        # train the best model using training and validation data combined
        trFeat = np.vstack((trainInput, vldInput))
        trTarget = np.vstack((trainOutput, vldOutput))

        print('Building opt ' + self.modelName)

        trainedModels = []
        for i in range(0, self.ensembleSize):
            # print('Ensembel member: ' +str(i+1)+'/'+str(ensembleSize))
            curSeed = self.randSeed + i * 10
            curModel = self.createTDCNN(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'],
                                 convActFunc=optParam['convActFunc'],
                                 pooling=optParam['pooling'], batchNorm=optParam['batchNorm'],
                                 dropoutRate=optParam['dropoutRate'],
                                 denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'],
                                 maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed)
            print(curModel.summary())
            
            curModel.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
            curModel.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'], verbose=0)

            trainedModels.append(curModel)

        self.trainedModels=trainedModels
        self.save()

        return
    
    def createTDCNN(self, convFilters, kernelSize, convActFunc, pooling, batchNorm, dropoutRate, denseNeurons, denseActFunc,
                  maxPredStep, numFeat, numChannel, randSeed=111):
        """
        :param convFilters:
        :param kernelSize:
        :param convActFunc:
        :param pooling:
        :param batchNorm:
        :param dropoutRate:
        :param denseNeurons:
        :param denseActFunc:
        :param maxPredStep:
        :param numFeat:
        :param numChannel:
        :param randSeed:
        :return:
        """
        model = Sequential()
        layerCount = -1
        for nFilters in convFilters:
            layerCount = layerCount + 1

            if layerCount == 0:
                model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=kernelSize, 
                                                 activation=convActFunc, padding='same'), input_shape=(numFeat, numChannel, 1)))
            else:
                model.add(
                    TimeDistributed(
                        Conv1D(filters=nFilters, kernel_size=kernelSize, activation=convActFunc, padding='same')))

            if pooling == 'max':
                model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))
            if batchNorm:
                model.add(TimeDistributed(BatchNormalization()))
            if dropoutRate:
                model.add(TimeDistributed(Dropout(dropoutRate)))

        model.add(Flatten())
        
        for neurons in denseNeurons:
            model.add(Dense(neurons, activation = denseActFunc))
        
        model.add(Dense(maxPredStep, activation = "linear"))

        return model
        
    # make prediction
    def predict(self, inputMatrix, histObsTime=None, deNormalize=False):
        """
        :param inputMatrix:
        :param predStartTime:
        :param deNormalize:
        :return:
        """
        # print('Status: Testing')
        if deNormalize:
            scalerFile=os.path.join(self.modelDir, 'scaler_target.pkl')
            dataScaler=pickle.load(open(scalerFile, 'rb'))

        numTestSample = inputMatrix.shape[0]
        allPred={self.modelName: np.empty((numTestSample, self.maxPredStep))}
        allPred[self.modelName][:] = np.nan

        if self.windowStart:
            windowStart = pd.to_datetime(arg=self.windowStart, format="%H:%M")
            windowStart = windowStart.replace(year=2000, month=1, day=1)
            windowEnd = pd.to_datetime(arg=self.windowEnd, format="%H:%M")
            windowEnd = windowEnd.replace(year=2000, month=1, day=1)

        for rowCount in range(numTestSample):
            feat=np.zeros(shape=(1, inputMatrix.shape[1], inputMatrix.shape[2]))
            feat[0,:,:] = inputMatrix[rowCount,:]
            predData = np.zeros((len(self.trainedModels), self.maxPredStep))

            for i in range(0, len(self.trainedModels)):
                curPred = self.trainedModels[i].predict(feat)
                curPred = curPred.flatten()

                # de-normalize the prediction if required
                if deNormalize:
                    curPred = curPred[:, np.newaxis]
                    curPred = dataScaler.inverse_transform(curPred)
                    curPred = curPred.flatten()

                curPred = np.clip(curPred, 0, None)
                predData[i, :] = curPred

            predData = np.nanmean(predData, axis=0)
            allPred[self.modelName][rowCount, :] = predData

            # manually set the predicted data as zero if prediction time is outside the specified window
            if self.windowStart:
                timeStart = histObsTime[rowCount]+pd.Timedelta(minutes=self.timeIncrement)
                timeEnd = timeStart + pd.Timedelta(minutes=self.timeIncrement * (self.maxPredStep - 1))
                timeRange = pd.Series(pd.date_range(start=timeStart, end=timeEnd, freq=str(self.timeIncrement) + 'min'))
                timeRange = timeRange.apply(lambda x: x.replace(year=2000, month=1, day=1))

                outsideWindowInd = list(timeRange.index[(timeRange < windowStart) | (timeRange > windowEnd)])
                allPred[self.modelName][rowCount, outsideWindowInd] = 0

        return allPred

    # load pre-trained model from disk
    def load(self):
        """
        :return:
        """
        from keras.models import load_model
        trainedModels=[]
        for i in range(self.ensembleSize):
            fileName=os.path.join(self.modelDir, self.modelName+'_'+str(i)+'.h5')
            if not os.path.isfile(fileName):
                raise FileNotFoundError('Model file does not exist')

            model=load_model(filepath=fileName)
            trainedModels.append(model)
            self.trainedModels=trainedModels

        return

    # save trained model to disk
    def save(self):
        """
        :return:
        """
        for i in range(self.ensembleSize):
            fileName = os.path.join(self.modelDir, self.modelName + '_' + str(i) + '.h5')
            self.trainedModels[i].save(fileName)

        return

## Time Distributed CNN LSTM

In [ ]:
class TDCNNLSTM:
    def __init__(self, config: dict):
        """
        :param config: a dict containing all config necessary for the model to operate
        """
        self.modelName='TDCNNLSTM' # model name
        self.configParam = config['models']['TDCNNLSTM'] # parameters
        self.loadSavedParam=config['loadSavedParam'] # used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
        self.ensembleSize=config['ensembleSize'] # ensemble size
        self.randSeed=config['randSeed'] # fix random seed for reproducibility

        self.modelDir=config['modelDir'] #dir for saving trained models/configuration, and/or extracting pre-trained models
        self.maxPredStep=config['maxPredStep'] # max prediction step

        if config['dataWindow']: # set the prediction outside this window as '0' regardless of the model's output.
            self.windowStart=config['dataWindow'][0]
            self.windowEnd=config['dataWindow'][1]
            self.timeIncrement=config['dataWindow'][2]
        else:
            self.windowStart=None
            self.windowEnd=None
            self.timeIncrement=None

        self.trainedModels=None
        
    def train(self, trainInput, trainOutput, vldInput, vldOutput, vldHisTime):
        """
        :param trainInput:
        :param trainOutput:
        :param vldInput:
        :param vldOutput:
        :param vldHisTime:
        :return:
        """

        print('Staus: Training   Model: ' + self.modelName)

        nFeat=trainInput.shape[1]
        nChannel=trainInput.shape[2]

        if self.loadSavedParam:
            paramFile = os.path.join(self.modelDir, 'opt_param_' + self.modelName + '.pkl')
            optParam = pickle.load(open(paramFile, 'rb'))
        else:
            modelCount = 0
            optParam = dict()
            for param in self.configParam:
                if not (param == 'searchParam'):
                    optParam[param] = copy.deepcopy(self.configParam[param][0])

            # build the initial model
            modelCount = modelCount + 1
            print(self.modelName + ' # ' + str(modelCount))

            model = self.createTDCNNLSTM(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'], convActFunc=optParam['convActFunc'],
                                      pooling=optParam['pooling'], batchNorm=optParam['batchNorm'], dropoutRate=optParam['dropoutRate'],
                                      denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'], lstmUnits = optParam['lstmUnits'],
                                      maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=self.randSeed)

            print(model.summary())
            
            # build subsequent models is required
            if self.configParam['searchParam']:
                # evaluate the previous model using validation data
                model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                model.fit(x=trainInput, y=trainOutput, batch_size=optParam['batchSize'], epochs=optParam['epochs'],
                             verbose=0)

                self.trainedModels=[model]

                predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                errDF.set_index('Steps', drop=True, inplace=True)
                optParam['bestValdAcc'] = errDF.loc['overall', 'RMSE']

                for param in self.configParam:
                    if param in ['searchParam', 'bestValdAcc']:
                        continue
                    convFilters = optParam['convFilters']
                    kernelSize = optParam['kernelSize']
                    convActFunc = optParam['convActFunc']
                    pooling = optParam['pooling']
                    batchNorm = optParam['batchNorm']
                    dropoutRate = optParam['dropoutRate']
                    denseNeurons = optParam['denseNeurons']
                    denseActFunc = optParam['denseActFunc']
                    batchSize = optParam['batchSize']
                    epochs = optParam['epochs']
                    vldSplit = optParam['vldSplit']
                    lstmUnits = optParam['lstmUnits']

                    for i in range(1, len(self.configParam[param])):
                        curParamValue = self.configParam[param][i]
                        if param == 'convFilters':
                            convFilters = curParamValue
                        elif param == 'kernelSize':
                            kernelSize = curParamValue
                        elif param == 'convActFunc':
                            convActFunc = curParamValue
                        elif param == 'pooling':
                            pooling = curParamValue
                        elif param == 'batchNorm':
                            batchNorm = curParamValue
                        elif param == 'dropoutRate':
                            dropoutRate = curParamValue
                        elif param == 'denseNeurons':
                            denseNeurons = curParamValue
                        elif param == 'denseActFunc':
                            denseActFunc = curParamValue
                        elif param == 'batchSize':
                            batchSize = curParamValue
                        elif param == 'epochs':
                            epochs = curParamValue
                        elif param == 'vldSplit':
                            vldSplit = curParamValue
                        elif param == 'lstmUnits':
                            lstmUnits = curParamValue

                        modelCount = modelCount + 1
                        model = self.createTDCNNLSTM(convFilters=convFilters, kernelSize=kernelSize, convActFunc=convActFunc,
                                          pooling=pooling, batchNorm=batchNorm, dropoutRate=dropoutRate,
                                          denseNeurons=denseNeurons, denseActFunc=denseActFunc, maxPredStep=self.maxPredStep,
                                          numFeat=nFeat, numChannel=nChannel, lstmUnits = lstmUnits, randSeed=self.randSeed)

                        model.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
                        model.fit(x=trainInput, y=trainOutput, batch_size=batchSize, epochs=epochs, verbose=0)
                        # previous best model
                        prevBestModel=self.trainedModels

                        # set the current model as the best model
                        self.trainedModels=[model]

                        # evaluate the previous model using validation data
                        predVldData = self.predict(inputMatrix=vldInput, histObsTime=vldHisTime, deNormalize=False)

                        errDF = computeAccuracy(actData=vldOutput, predData=predVldData[self.modelName])
                        errDF.set_index('Steps', drop=True, inplace=True)
                        curMetric = errDF.loc['overall', 'RMSE']

                        if curMetric < optParam['bestValdAcc']:
                            optParam[param] = curParamValue
                            optParam['bestValdAcc'] = curMetric
                        else:
                            self.trainedModels=prevBestModel # restore the previous best model

                # save the best parameters in file
                pickle.dump(optParam,open(os.path.join(self.modelDir, 'opt_param_' + self.modelName+ '.pkl'), 'wb'))

        # train the best model using training and validation data combined
        trFeat = np.vstack((trainInput, vldInput))
        trTarget = np.vstack((trainOutput, vldOutput))

        print('Building opt ' + self.modelName)

        trainedModels = []
        for i in range(0, self.ensembleSize):
            # print('Ensembel member: ' +str(i+1)+'/'+str(ensembleSize))
            curSeed = self.randSeed + i * 10
            curModel = self.createTDCNNLSTM(convFilters=optParam['convFilters'], kernelSize=optParam['kernelSize'],
                                 convActFunc=optParam['convActFunc'],
                                 pooling=optParam['pooling'], batchNorm=optParam['batchNorm'],
                                 dropoutRate=optParam['dropoutRate'],
                                 denseNeurons=optParam['denseNeurons'], denseActFunc=optParam['denseActFunc'], lstmUnits = optParam['lstmUnits'],
                                 maxPredStep=self.maxPredStep, numFeat=nFeat, numChannel=nChannel, randSeed=curSeed)

            curModel.compile(optimizer='adamax', loss='mean_squared_error', metrics=['mean_squared_error'])
            curModel.fit(x=trFeat, y=trTarget, batch_size=optParam['batchSize'], epochs=optParam['epochs'], verbose=0)

            trainedModels.append(curModel)

        self.trainedModels=trainedModels
        self.save()

        return
    
    def createTDCNNLSTM(self, convFilters, kernelSize, convActFunc, pooling, batchNorm, dropoutRate, denseNeurons, denseActFunc,
                  maxPredStep, numFeat, numChannel, lstmUnits, randSeed=111):

        model = Sequential()
        layerCount = -1
        for nFilters in convFilters:
            layerCount = layerCount + 1

            if layerCount == 0:
                model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=kernelSize, 
                                                 activation=convActFunc, padding='same'), input_shape=(numFeat, numChannel, 1)))
            else:
                model.add(
                    TimeDistributed(
                        Conv1D(filters=nFilters, kernel_size=kernelSize, activation=convActFunc, padding='same')))

            if pooling == 'max':
                model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))
            if batchNorm:
                model.add(TimeDistributed(BatchNormalization()))
            if dropoutRate:
                model.add(TimeDistributed(Dropout(dropoutRate)))

        model.add(TimeDistributed(Flatten()))
        
        model.add(Bidirectional(LSTM(units=lstmUnits, return_sequences=False, activation='tanh')))
        
        for neurons in denseNeurons:
            model.add(Dense(neurons, activation = denseActFunc))
        
        model.add(Dense(maxPredStep, activation = "linear"))

        return model
        
    # make prediction
    def predict(self, inputMatrix, histObsTime=None, deNormalize=False):
        """
        :param inputMatrix:
        :param predStartTime:
        :param deNormalize:
        :return:
        """
        # print('Status: Testing')
        if deNormalize:
            scalerFile=os.path.join(self.modelDir, 'scaler_target.pkl')
            dataScaler=pickle.load(open(scalerFile, 'rb'))

        numTestSample = inputMatrix.shape[0]
        allPred={self.modelName: np.empty((numTestSample, self.maxPredStep))}
        allPred[self.modelName][:] = np.nan

        if self.windowStart:
            windowStart = pd.to_datetime(arg=self.windowStart, format="%H:%M")
            windowStart = windowStart.replace(year=2000, month=1, day=1)
            windowEnd = pd.to_datetime(arg=self.windowEnd, format="%H:%M")
            windowEnd = windowEnd.replace(year=2000, month=1, day=1)

        for rowCount in range(numTestSample):
            feat=np.zeros(shape=(1, inputMatrix.shape[1], inputMatrix.shape[2]))
            feat[0,:,:] = inputMatrix[rowCount,:]
            predData = np.zeros((len(self.trainedModels), self.maxPredStep))

            for i in range(0, len(self.trainedModels)):
                curPred = self.trainedModels[i].predict(feat)
                curPred = curPred.flatten()

                # de-normalize the prediction if required
                if deNormalize:
                    curPred = curPred[:, np.newaxis]
                    curPred = dataScaler.inverse_transform(curPred)
                    curPred = curPred.flatten()

                curPred = np.clip(curPred, 0, None)
                predData[i, :] = curPred

            predData = np.nanmean(predData, axis=0)
            allPred[self.modelName][rowCount, :] = predData

            # manually set the predicted data as zero if prediction time is outside the specified window
            if self.windowStart:
                timeStart = histObsTime[rowCount]+pd.Timedelta(minutes=self.timeIncrement)
                timeEnd = timeStart + pd.Timedelta(minutes=self.timeIncrement * (self.maxPredStep - 1))
                timeRange = pd.Series(pd.date_range(start=timeStart, end=timeEnd, freq=str(self.timeIncrement) + 'min'))
                timeRange = timeRange.apply(lambda x: x.replace(year=2000, month=1, day=1))

                outsideWindowInd = list(timeRange.index[(timeRange < windowStart) | (timeRange > windowEnd)])
                allPred[self.modelName][rowCount, outsideWindowInd] = 0

        return allPred

    # load pre-trained model from disk
    def load(self):
        """
        :return:
        """
        from keras.models import load_model
        trainedModels=[]
        for i in range(self.ensembleSize):
            fileName=os.path.join(self.modelDir, self.modelName+'_'+str(i)+'.h5')
            if not os.path.isfile(fileName):
                raise FileNotFoundError('Model file does not exist')

            model=load_model(filepath=fileName)
            trainedModels.append(model)
            self.trainedModels=trainedModels

        return

    # save trained model to disk
    def save(self):
        """
        :return:
        """
        for i in range(self.ensembleSize):
            fileName = os.path.join(self.modelDir, self.modelName + '_' + str(i) + '.h5')
            self.trainedModels[i].save(fileName)

        return

# TRAINING AND TESTING

## Model Configuration

In [7]:
modelConfig={
    'models':{
        'CNN':{
            'searchParam': False,
            'convFilters':[[32, 16]],
            'kernelSize':[3],
            'convActFunc':['tanh'],
            'pooling':['max'],
            'batchNorm':[False],
            'dropoutRate':[0.2],
            'denseNeurons':[[20]],
            'denseActFunc':['sigmoid'],
            'batchSize':[128],
            'vldSplit':[0.1], 'epochs':[500]
        }
#         'LSTM':{
#             'searchParam': False,
#             'lstmBlocks':[[64, 32]],
#             'lstmActFunc':['relu','tanh', 'sigmoid'],
#             'recActFunc':['sigmoid', 'tanh'],
#             'batchNorm': [False],
#             'dropoutRate': [0.1, 0],
#             'recDropoutRate': [0],
#             'denseNeurons': [[48]],
#             'denseActFunc':['relu', 'sigmoid', 'tanh'],
#             'batchSize':[128, 512],
#             'vldSplit': [0.1],
#             'epochs':[500, 1000]
#         }
#         'TCN': {
#            'searchParam' : True, 
#            'nbFilters' : [[64], [64, 32], [32], [32, 16]], 
#            'kernelSize' : [2, 3], 
#            'dilations' : [[1,2,4], [1,2,4,8], [1, 2, 4, 8, 16]],
#            'kernel_initialiser' : ['he_normal', 'glorot_uniform'], 
#            'dropoutRate' : [0, 0.05], 
#            'denseNeurons':[[20], [48]], 
#            'batchNorm' : [False, True], 
#            'denseActFunc':['sigmoid'], 
#            'batchSize':[128, 512],
#            'vldSplit': [0.1],
#            'epochs':[500, 1000]
#          },
#         'TCN': {
#            'searchParam' : False, 
#            'nbFilters' : [[32, 16]], 
#            'kernelSize' : [3], 
#            'dilations' : [[1,2,4]],
#            'kernel_initialiser' : ['he_normal'], 
#            'dropoutRate' : [0, 0.05], 
#            'denseNeurons':[[20], [48]], 
#            'batchNorm' : [False, True], 
#            'denseActFunc':['sigmoid'], 
#            'batchSize':[128, 512],
#            'vldSplit': [0.1],
#            'epochs':[500, 1000]
#          },
#         'TCAN': {
#            'searchParam' : False, 
#            'nbFilters' : [[64], [64, 32], [32], [32, 16]], 
#            'kernelSize' : [2, 3], 
#            'dilations' : [[1,2,4], [1,2,4,8], [1, 2, 4, 8, 16]],
#            'kernel_initialiser' : ['he_normal', 'glorot_uniform'], 
#            'dropoutRate' : [0, 0.05], 
#            'denseNeurons':[[20], [48]], 
#            'batchNorm' : [False, True], 
#            'denseActFunc':['sigmoid'], 
#            'batchSize':[128, 512],
#            'vldSplit': [0.1],
#            'epochs':[1, 500, 1000]
#         },
#         'TDCNNLSTM':{
#             'searchParam': True,
#             'convFilters':[[32], [64], [32, 16]],
#             'kernelSize':[3, 5],
#             'convActFunc':['relu', 'tanh'],
#             'pooling':['max', None],
#             'batchNorm':[False, True],
#             'dropoutRate':[0.2, 0],
#             'denseNeurons':[[20], [48]],
#             'denseActFunc':['sigmoid', 'relu', 'tanh'],
#             'batchSize':[48, 96],
#             'lstmUnits' : [64, 128],
#             'vldSplit':[0.1], 'epochs':[500, 1000]
#         }
#         'TDCNN': {
#             'searchParam': True,
#             'convFilters':[[32], [64], [32, 16]],
#             'kernelSize':[3,5],
#             'convActFunc':['relu', 'tanh'],
#             'pooling':['max', None],
#             'batchNorm':[False, True],
#             'dropoutRate':[0.2, 0],
#             'denseNeurons':[[20],[48]],
#             'denseActFunc':['relu', 'tanh'],
#             'batchSize':[48, 96],
#             'epochs':[500, 1000], 
#             'vldSplit': [0.1]
#         }
#         'TDCNNLSTM':{
#             'searchParam': False,
#             'convFilters':[[64]],
#             'kernelSize':[5],
#             'convActFunc':['tanh'],
#             'pooling':['max'],
#             'batchNorm':[True],
#             'dropoutRate':[0.2],
#             'denseNeurons':[[20]],
#             'denseActFunc':['sigmoid'],
#             'batchSize':[48],
#             'lstmUnits' : [64],
#             'vldSplit':[0.1], 'epochs':[500]
#         },
#         'Benchmark1': {
#             'searchParam': False,
#             'convFilters':[[32], [64], [32, 16]],
#             'kernelSize':[3,5],
#             'convActFunc':['relu', 'tanh'],
#             'pooling':['max', None],
#             'batchNorm':[False, True],
#             'dropoutRate':[0.2, 0],
#             'denseNeurons':[[20],[48]],
#             'denseActFunc':['relu', 'tanh'],
#             'batchSize':[48, 96],
#             'epochs':[500, 1000], 
#             'vldSplit': [0.1]
#          },
#         'Benchmark2': {
#             'searchParam': False,
#             'convFilters':[[32], [64], [32, 16]],
#             'kernelSize':[3,5],
#             'convActFunc':['relu', 'tanh'],
#             'pooling':['max', None],
#             'batchNorm':[False, True],
#             'dropoutRate':[0.2, 0],
#             'denseNeurons':[[20],[48]],
#             'denseActFunc':['relu', 'tanh'],
#             'batchSize':[48, 96],
#             'epochs':[500, 1000], 
#             'vldSplit': [0.1]        
#    }
    },
    'loadSavedParam': False, #  used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
    'ensembleSize': 10, # ensemble size
    'dataWindow': ['07:00', '19:00'],  # set the prediction outside this window as '0' regardless of the model's output.
}

## Parameter Summary

In [ ]:
modelConfig={
    'models':{        
#         'CNN':{
#             'searchParam': False,
#             'convFilters':[[32, 16]],
#             'kernelSize':[3],
#             'convActFunc':['tanh'],
#             'pooling':['max'],
#             'batchNorm':[False],
#             'dropoutRate':[0.2],
#             'denseNeurons':[[20]],
#             'denseActFunc':['sigmoid'],
#             'batchSize':[128],
#             'vldSplit':[0.1], 'epochs':[500]
#         }
#         'LSTM':{
#             'searchParam': False,
#             'lstmBlocks':[[64, 32]],
#             'lstmActFunc':['relu', 'tanh', 'sigmoid'],
#             'recActFunc':['sigmoid', 'tanh'],
#             'batchNorm': [False],
#             'dropoutRate': [0.1, 0],
#             'recDropoutRate': [0],
#             'denseNeurons': [[1]],
#             'denseActFunc':['relu', 'sigmoid', 'tanh'],
#             'batchSize':[128, 512],
#             'vldSplit': [0.1],
#             'epochs':[500, 1000]
#         }
#         'TCN': { #greenland
#            'searchParam' : False, 
#            'nbFilters' : [[32, 16]], 
#            'kernelSize' : [3], 
#            'dilations' : [[1,2,4]],
#            'kernel_initialiser' : ['he_normal'], 
#            'dropoutRate' : [0, 0.05], 
#            'denseNeurons':[[20], [48]], 
#            'batchNorm' : [False, True], 
#            'denseActFunc':['sigmoid'], 
#            'batchSize':[128, 512],
#            'vldSplit': [0.1],
#            'epochs':[500, 1000]
#          }
#         'TCN': { #chromasun
#            'searchParam' : False, 
#            'nbFilters' : [[32, 16]], 
#            'kernelSize' : [2], 
#            'dilations' : [[1,2,4, 8]],
#            'kernel_initialiser' : ['he_normal'], 
#            'dropoutRate' : [0.05], 
#            'denseNeurons':[[48]], 
#            'batchNorm' : [True], 
#            'denseActFunc':['sigmoid'], 
#            'batchSize':[512],
#            'vldSplit': [0.1],
#            'epochs':[500, 1000]
#          }
#         'TDCNNLSTM':{
#             'searchParam': False,
#             'convFilters':[[64]],
#             'kernelSize':[5],
#             'convActFunc':['tanh'],
#             'pooling':['max'],
#             'batchNorm':[True],
#             'dropoutRate':[0.2],
#             'denseNeurons':[[20]],
#             'denseActFunc':['sigmoid'],
#             'batchSize':[48],
#             'lstmUnits' : [64],
#             'vldSplit':[0.1], 'epochs':[500]
#         }
        'TDCNN': {
            'searchParam': False,
            'convFilters':[[32], [64], [32, 16]],
            'kernelSize':[3,5],
            'convActFunc':['relu', 'tanh'],
            'pooling':['max', None],
            'batchNorm':[False, True],
            'dropoutRate':[0.2, 0],
            'denseNeurons':[[20],[48]],
            'denseActFunc':['tanh'],
            'batchSize':[96],
            'epochs':[500, 1000], 
            'vldSplit': [0.1]
        }
#           'TDCNN': {  #greenland
#             'searchParam': False,
#             'convFilters':[[32, 16]],
#             'kernelSize':[3,5],
#             'convActFunc':['relu', 'tanh'],
#             'pooling':['max', None],
#             'batchNorm':[False, True],
#             'dropoutRate':[0.2, 0],
#             'denseNeurons':[[20],[48]],
#             'denseActFunc':['tanh'],
#             'batchSize':[96],
#             'epochs':[500, 1000], 
#             'vldSplit': [0.1]
#         }
    },
    'loadSavedParam': False, #  used only when is_Training=True. loadSavedParam=True: load the saved optimal param and then train the model; loadSavedParam=False: search the optimal param
    'ensembleSize': 10, # ensemble size
    'dataWindow': ['07:00', '19:00'],  # set the prediction outside this window as '0' regardless of the model's output.
}

## Dataset Choice

### Chromasun

In [15]:
noisyData = False


expConfig={
    # data set identifier
    'dataSetName': 'Chromasun', # data set/solar collector name

    # directories
    'dataDir': os.path.join(os.getcwd(), 'Data'),  # directory containing the data from all sources
    'modelDir': os.path.join(os.getcwd(), 'Models'), #dir for saving trained models/configuration, and/or extracting pre-trained models
    'resDir': os.path.join(os.getcwd(), 'Results'),  # dir to save the prediction results/accuracy

    # info on site data
    'site_data_file':'Chromasun_raw_data.csv', # csv/xl file containing the historical raw data (including Ambient Temp) from site
    'baseResolution': 5, # resolution (in minutes) of the raw data obtained from site
    'site_datetime_format':'%d/%m/%Y %H:%M', # format of date (in NSW local time zone) for site data
    'dataExclusion':['01-01-2020'], # only during training/retraining exclude data for the date range specified. if only single date is provided, exclude data for all days forward starting that date
    'waterSpecHeat': 4.208, # this is require to compute Flow rate from raw data
    # name of the columns expected in site_data_file and how to rename them later
    'colAlias':{'Date/Time': 'Time', 'HTHHW IN - Output': 'Temp_output', 'HTHHW OUT - Output': 'Temp_input',
                'PU-07 HTHHW FS - State': 'PU_FS', 'HTHHW FM - Output': 'HT_FM',
                'CH-01 Run Status - State':'Chiller_state', 'Ambient Temp': 'Amb_temp'},

    # information on processed/cleaned site data
    'resolution': 30,  # desired resolution (in minutes) of site data after cleaning/processing
    'targetCol': 'Power_output',  # name of the target col
    'dateCol': 'Time',  # name of the col containing timestamp
    'inletTempCol': 'Temp_input',  # name of the col containing inlet fluid temperature
    'outletTempCol': 'Temp_output',  # name of the col containing outlet fluid temperature
    'chillerStateCol': 'Chiller_state',  # name of the col containing the chiller's state
    'ambTempCol': 'Amb_temp',  # name of the col containing ambient temperature
    'flowrateCol': 'Flow_rate',  # name of the col containing flow rate
    'deltaTempCol': 'Delta_temp',  # name of the col containing temperature difference (outlet temp - inlet temp)

    # parameters for data cleaning and outliers detection
    'dataWindow': ['07:00', '19:00'],  # check data quality (e.g. missing samples) only within this window of time for each day
    'chillerOffThreshold': 80,  # accepted highest percentage of chiller off state within dataWindow for each day
    'percentMissingThreshold': 10,  # accepted highest percentage of missing data within dataWindow for each day
    'percentZeroThreshold': 95,  # accepted highest percentage of zero valued data within dataWindow for each day
    'minFRthreshold':3, # min threshold for flow rate
    'frWindowLen':15, # time (in minutes) to check the flow rate continuation for manual outliers detection
    'maxPowerThreshold':500, # maximum possible power output from the solar collector
    'minPowerThreshold':20, # minimum possible power output (this is required for detecting days with only few small spikes)
    'outliersDetMethods':{'LOF':{'contamination':'auto'}}, # configuration parameters for outlier detection method

    # information on historical GHI/DNI data
    'hist_GHI_file':'hist_GHI.csv',  # csv file containing historical GHI
    'hist_DNI_file':'hist_DNI.csv',  # csv file containing historical DNI
    'hist_GHI_DNI_resolution':60, # resolution (in minutes) of histoircal GHI/DNI data
    'hist_GHI_DNI_dateCol':'datetime', # column name  containing the date in hist_GHI_file and hist_DNI_file
    'hist_GHI_DNI_valueCol': ' value', # column name  containing the GHI/DNI value in hist_GHI_file/hist_DNI_file
    'hist_GHI_DNI_datetime_format':'%Y%m%d%H', # format of date in hist_GHI_file/hist_DNI_file
    'hist_GHI_DNI_timezone':'utc', # time zone of date (either 'utc, or 'local' or None. 'local'/None means NSW timezone) in hist_GHI_file/hist_DNI_file

    # information on predicted GHI/DNI/Amb_temp data
    'pred_data_file': None,  # csv file containing predicted GHI/DNI/Amb_temp for future (required if is_training=False)
    'pred_data_resolution': 30, # resolution (in minutes) of predicted weather data (GHI/DNI/Amb temp) for future
    'pred_data_dateCol':'Date/Time',  # column name  containing the date in pred_data_file
    'pred_data_datetime_format':'%d/%m/%Y %H:%M',# '%Y%m%d%H%M',  # format of date in pred_data_file, all date in local time zone
    'pred_GHI_valueCol':'pred_GHI', # column name containing the predicted GHI data in pred_data_file
    'pred_DNI_valueCol':'pred_DNI', # column name containing the predicted DNI data in pred_data_file
    'pred_Amb_temp_valueCol':'pred_Amb_temp', # column name containing the predicted Amb temp data in pred_data_file
    'pred_Chiller_state_valueCol':'pred_Chiller_state', # column name containing the predicted Chiller state data in pred_data_file

    # how to split and shuffle (if required) before fitting data to the prediction models
    'dataSplitInfo': {'train': 50, 'vld': 25, 'test': 25}, # train-validation-test split
    'doShuffle': False,# whether to shuffle the days before split
    'randSeed': 111, # random state for shuffling to reproduce the results

    # parameter for data normalization
    'scalingParam': {'minValue': 0, 'maxValue': 1},  # how to normalize the load and other exogenous variables

    # specify prediction horizon and inputs for prediction models
    'maxPredStep': 48,  # setup prediction horizon, 48 means up to 24 hours since the cleaned data in 30 minutes intervals
    'inputs': { # list of input variables to be used for the prediction models
        'nLagDay': 1,  # num of previous lags (in terms of day) to use as input to the models; 1: last 48 values, 2: last 96 values
        'histVar': ['GHI', 'DNI', 'Amb_temp'],  # historical input variables
        'predVar': [],#['pred_GHI', 'pred_DNI', 'pred_Amb_temp'],  # predicted weather data for future
        'calendarVar': [],#['DoY', 'ToD'],  # calendar inputs (DoY: day of the year; 'ToD': time of the day)
    }
}


is_training = True
model = Forecastmodel(expConfig=expConfig, modelConfig=modelConfig)
combData = model.preprocess_chromasun(is_training=is_training, noisyData = noisyData)

Data preparation is in progress........, please wait


### Greenland

In [10]:
noisyData = False


expConfig={
    # data set identifier
    'dataSetName': 'Greenland_NOISE', # data set/solar collector name

    # directories
    'dataDir': os.path.join(os.path.dirname(os.getcwd()), 'RH/Data'),  # directory containing the data from all sources
    'modelDir': os.path.join(os.getcwd(), 'Models'), #dir for saving trained models/configuration, and/or extracting pre-trained models
    'resDir': os.path.join(os.getcwd(), 'Results'),  # dir to save the prediction results/accuracy

    # info on site data
    'site_data_file':'Greenland_raw_data.csv', # csv/xlsx file containing the historical raw data (including Ambient Temp) from site
    'baseResolution': 5, # resolution (in minutes) of the raw data obtained from site
    'site_datetime_format':'%d/%m/%Y %H:%M', # format of date (in NSW local time zone) for site data
    'dataExclusion':[], # only during training/retraining exclude data for the date range specified. if only single date is provided, exclude data for all days forward starting that date
    'waterSpecHeat': 4.19, # this is require to power output from raw data
    # name of the columns expected in site_data_file and how to rename them later
    'colAlias':{'Row Labels': 'Time', 'T1 SF - Collector Out': 'Temp_output', 'T2 SF - Collector In': 'Temp_input',
                'FM1 9 l/s span - Collector Flow': 'Flow_rate', 'Amb Temp': 'Amb_temp'},

    # information on processed/cleaned site data
    'resolution': 30,  # desired resolution (in minutes) of site data after cleaning/processing
    'targetCol': 'Power_output',  # name of the target col
    'dateCol': 'Time',  # name of the col containing timestamp
    'inletTempCol': 'Temp_input',  # name of the col containing inlet fluid temperature
    'outletTempCol': 'Temp_output',  # name of the col containing outlet fluid temperature
    'chillerStateCol': 'Chiller_state',  # name of the col containing the chiller's state
    'ambTempCol': 'Amb_temp',  # name of the col containing ambient temperature
    'flowrateCol': 'Flow_rate',  # name of the col containing flow rate
    'deltaTempCol': 'Delta_temp',  # name of the col containing temperature difference (outlet temp - inlet temp)

    # parameters for data cleaning and outliers detection
    'dataWindow': ['07:00', '19:00'],  # check data quality (e.g. missing samples) only within this window of time for each day
    'chillerOffThreshold': 80,  # accepted highest percentage of chiller off state within dataWindow for each day
    'percentMissingThreshold': 5,  # accepted highest percentage of missing data within dataWindow for each day
    'percentZeroThreshold': 95,  # accepted highest percentage of zero valued data within dataWindow for each day
    'minFRthreshold':3, # min threshold for flow rate
    'frWindowLen':15, # time (in minutes) to check the flow rate continuation for manual outliers detection
    'maxPowerThreshold':500, # maximum possible power output from the solar collector
    'minPowerThreshold':20, # minimum possible power output (this is required for detecting days with only few small spikes)
    'outliersDetMethods':{'LOF':{'contamination':'auto'}}, # configuration parameters for outlier detection method


    # information on historical GHI/DNI data
    'hist_GHI_file':'hist_GHI.csv',  # csv file containing historical GHI
    'hist_DNI_file':'hist_DNI.csv',  # csv file containing historical DNI
    'hist_GHI_DNI_resolution':60, # resolution (in minutes) of histoircal GHI/DNI data
    'hist_GHI_DNI_dateCol':'datetime', # column name  containing the date in hist_GHI_file and hist_DNI_file
    'hist_GHI_DNI_valueCol': ' value', # column name  containing the GHI/DNI value in hist_GHI_file/hist_DNI_file
    'hist_GHI_DNI_datetime_format':'%Y%m%d%H', # format of date in hist_GHI_file/hist_DNI_file
    'hist_GHI_DNI_timezone':'utc', # time zone of date (either 'utc, or 'local' or None. 'local'/None means NSW timezone) in hist_GHI_file/hist_DNI_file

    # information on predicted GHI/DNI/Amb_temp data
    'pred_data_file': None,  # csv file containing predicted GHI/DNI/Amb_temp for future (required if is_training=False)
    'pred_data_resolution': 30, # resolution (in minutes) of predicted weather data (GHI/DNI/Amb temp) for future
    'pred_data_dateCol':'Date/Time',  # column name  containing the date in pred_data_file
    'pred_data_datetime_format':'%d/%m/%Y %H:%M',# '%Y%m%d%H%M',  # format of date in pred_data_file, all date in local time zone
    'pred_GHI_valueCol':'pred_GHI', # column name containing the predicted GHI data in pred_data_file
    'pred_DNI_valueCol':'pred_DNI', # column name containing the predicted DNI data in pred_data_file
    'pred_Amb_temp_valueCol':'pred_Amb_temp', # column name containing the predicted Amb temp data in pred_data_file
    'pred_Chiller_state_valueCol':'pred_Chiller_state', # column name containing the predicted Chiller state data in pred_data_file

    # how to split and shuffle (if required) before fitting data to the prediction models
    'dataSplitInfo': {'train': 50, 'vld': 25, 'test': 25}, # train-validation-test split
    'doShuffle': False,# whether to shuffle the days before split
    'randSeed': 111, # random state for shuffling to reproduce the results

    # parameter for data normalization
    'scalingParam': {'minValue': 0, 'maxValue': 1},  # how to normalize the load and other exogenous variables

    # specify prediction horizon and inputs for prediction models
    'maxPredStep': 48,  # setup prediction horizon, 48 means up to 24 hours since the cleaned data in 30 minutes intervals
    'inputs': { # list of input variables to be used for the prediction models
        'nLagDay': 1,  # num of previous lags (in terms of day) to use as input to the models; 1: last 48 values, 2: last 96 values
        'histVar': [],#['GHI', 'DNI', 'Amb_temp'],  # historical input variables
        'predVar': [], #['pred_GHI', 'pred_DNI', 'pred_Amb_temp'],  # predicted weather data for future
        'calendarVar': ['DoY', 'ToD'],  # calendar inputs (DoY: day of the year; 'ToD': time of the day)
    }
}


is_training = True
model = Forecastmodel(expConfig=expConfig, modelConfig=modelConfig)
combData = model.preprocess_greenland(is_training=is_training, noisyData = noisyData) 

Data preparation is in progress........, please wait


## Training

In [16]:
if is_training:
    model.train(orgData=combData)
else:
    predData=model.predict(orgData=combData)

print('Program finished successfully !')

Preparing IO examples for training data set
(5328, 48, 4)
(5328, 48)
Preparing IO examples for validation data set
(2688, 48, 4)
(2688, 48)
Preparing IO examples for testing data set
(2592, 48, 4)
(2592, 48)
Staus: Training   Model: CNN
CNN # 1
Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_66 (Conv1D)          (None, 48, 32)            416       
                                                                 
 max_pooling1d_66 (MaxPooli  (None, 24, 32)            0         
 ng1D)                                                           
                                                                 
 dropout_66 (Dropout)        (None, 24, 32)            0         
                                                                 
 conv1d_67 (Conv1D)          (None, 24, 16)            1552      
                                                                 
 max_p

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step
Program finished successfully !


In [ ]:
obj = pd.read_pickle('Results/TCN Greenland Noise/opt_param_TCN.pkl')
obj